In [1]:
import sklearn
print (sklearn.__version__)


1.0.2


In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter
import itertools
import pickle
import seaborn as sn
import sklearn as sk
import xgboost
import sklearn
import openml


from xgboost import plot_importance
from modAL.models import ActiveLearner, Committee
from modAL.uncertainty import uncertainty_sampling, entropy_sampling
from modAL.expected_error import expected_error_reduction
from modAL.density import information_density
from modAL.disagreement import vote_entropy_sampling, consensus_entropy_sampling, max_disagreement_sampling
from modAL.utils.selection import multi_argmax, shuffled_argmax
from numpy import trapz, log2


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import clone
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score
from sklearn.metrics.pairwise import pairwise_kernels, KERNEL_PARAMS
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.utils import check_array

from scipy import stats
from scipy.special import gammaln, factorial
from scipy.spatial.distance import cdist

from openml.datasets import edit_dataset, fork_dataset, get_dataset

# from pip command
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import os
import warnings
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from copy import deepcopy
import multiprocessing

warnings.filterwarnings('ignore') # precision often has cases where there are no predictions for the minority class, which leads to zero-division. This keeps that warning from showing up.

In [5]:
openml_list = openml.datasets.list_datasets()  # returns a dict

# Show a nice table with some key data properties
datalist = pd.DataFrame.from_dict(openml_list, orient="index")
datalist = datalist[["did", "name", "NumberOfInstances", "NumberOfFeatures", "NumberOfClasses"]]

print(f"First 10 of {len(datalist)} datasets...")
datalist.head(n=10)

# The same can be done with lesser lines of code
openml_df = openml.datasets.list_datasets(output_format="dataframe")
openml_df.head(n=10)

First 10 of 4040 datasets...


,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
7,7,audiology,1,1,active,ARFF,57.0,24.0,1.0,24.0,70.0,226.0,222.0,317.0,0.0,70.0
8,8,liver-disorders,1,1,active,ARFF,NaN,NaN,NaN,0.0,6.0,345.0,0.0,0.0,6.0,0.0
9,9,autos,1,1,active,ARFF,67.0,22.0,3.0,6.0,26.0,205.0,46.0,59.0,15.0,11.0
10,10,lymph,1,1,active,ARFF,81.0,8.0,2.0,4.0,19.0,148.0,0.0,0.0,3.0,16.0
11,11,balance-scale,1,1,active,ARFF,288.0,3.0,49.0,3.0,5.0,625.0,0.0,0.0,4.0,1.0


In [6]:
datalist[datalist.NumberOfInstances > 10000].sort_values(["NumberOfInstances"]).head(n=20)
""
datalist.query('name == "credit-g"')


,did,name,NumberOfInstances,NumberOfFeatures,NumberOfClasses
31,31,credit-g,1000.0,21.0,2.0


In [5]:
# This is done based on the dataset ID.
dataset = openml.datasets.get_dataset(31) # Get credit dataset

# Print a summary
print(
    f"This is dataset '{dataset.name}', the target feature is "
    f"'{dataset.default_target_attribute}'"
)
print(f"URL: {dataset.url}")
print(dataset.description[:500])

This is dataset 'credit-g', the target feature is 'class'
URL: https://old.openml.org/data/v1/download/31/credit-g.arff
**Author**: Dr. Hans Hofmann  
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    
**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)

**German Credit dataset**  
This dataset classifies people described by a set of attributes as good or bad credit risks.

This dataset comes with a cost matrix: 
``` 
Good  Bad (predicted)  
Good   0    1 (actual)  
Bad    5    0  
```

It is worse to class a customer as good when they are ba


In [6]:
X, y, categorical_indicator, attribute_names = dataset.get_data(
    target=dataset.default_target_attribute, dataset_format="dataframe"
)
print(X.head())
print(X.info())

  checking_status  duration                  credit_history  \
0              <0         6  critical/other existing credit   
1        0<=X<200        48                   existing paid   
2     no checking        12  critical/other existing credit   
3              <0        42                   existing paid   
4              <0        24              delayed previously   

               purpose  credit_amount    savings_status employment  \
0             radio/tv         1169.0  no known savings        >=7   
1             radio/tv         5951.0              <100     1<=X<4   
2            education         2096.0              <100     4<=X<7   
3  furniture/equipment         7882.0              <100     4<=X<7   
4              new car         4870.0              <100     1<=X<4   

   installment_commitment     personal_status other_parties  residence_since  \
0                       4         male single          none                4   
1                       2  female div/de

In [7]:
run_openml_test(data=X,k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

NameError: name 'run_openml_test' is not defined

In [18]:
# Setting up a density-weighted sampling method given a classifier and a candidate pool of instances to be sampled. 
# By default only one instance is sampled. Multi-instance sampling not supported.
def density_sampling(classifier, X_pool, n_instances: int = 1, **predict_proba_kwargs):
    density = information_density(X_pool, "cosine") # cosine or euclidean
    density = np.ones(len(density)) - density
    
    try:
        classwise_uncertainty = classifier.predict_proba(X_pool, **predict_proba_kwargs)
    except NotFittedError:
        return np.ones(shape=(X.shape[0], ))

    # for each point, select the maximum uncertainty
    uncertainty = 1 - np.max(classwise_uncertainty, axis=1)
    dense_informative = np.multiply(uncertainty, density)

    query_idx = shuffled_argmax(dense_informative, n_instances=n_instances)
    return query_idx, X_pool[query_idx]

In [19]:
# Empty method which allows for the creation of committee for QBC sampling while using the same style of AL selection as other methods.
def qbc_sampling(classifier, X_pool):
    return 0, 0

In [26]:
# Setting up a random sampling strategy given a classifier and a candidate pool of instances to be sampled.
def random_sampling(classifier, X_pool):
    n_samples = len(X_pool)
    query_idx = np.random.choice(range(n_samples))
    return query_idx, X_pool[query_idx]

In [20]:
# Helper functions for the different versions of PAL

def euler_beta(a, axis=1):
    """
    Represents Euler beta function: B(a(i)) = Gamma(a(i,1))*...*Gamma(a_n)/Gamma(a(i,1)+...+a(i,n))
    Parameters
    ----------
    a: array-like, shape (m, n)
        Vectors to evaluated.
    axis: int
        Determines along which axis the Euler beta function is computed.
    Returns
    -------
    result: array-like, shape (m)
        Euler beta function results [B(a(0)), ..., B(a(m))
    """
    return np.exp(np.sum(gammaln(a), axis=axis)-gammaln(np.sum(a, axis=axis)))

def multinomial_coefficient(a, axis=1):
    """
    Computes Multinomial coefficient: Mult(a(i)) = (a(i,1)+...+a(i,n))!/(a(i,1)!...a(i,n)!)
    Parameters
    ----------
    a: array-like, shape (m, n)
        Vectors to evaluated.
    axis: int
        Determines along which axis the Euler beta function is computed.
    Returns
    -------
    result: array-like, shape (m)
        Multinomial coefficients [Mult(a(0)), ..., Mult(a(m))
    """
    return factorial(np.sum(a, axis=axis))/np.prod(factorial(a), axis=axis)

def gen_l_vec_list(m_approx, n_classes):
    """
    Creates all possible class labeling vectors for given number of hypothetically acquired labels and given number of
    classes.
    Parameters
    ----------
    m_approx: int
        Number of hypothetically acquired labels..
    n_classes: int,
        Number of classes
    Returns
    -------
    label_vec_list: array-like, shape = [n_label_vectors, n_classes]
        All possible label vectors for given parameters.
    """

    label_vec_list = [[]]
    label_vec_res = np.arange(m_approx + 1)
    for i in range(n_classes - 1):
        new_label_vec_list = []
        for labelVec in label_vec_list:
            for newLabel in label_vec_res[label_vec_res - (m_approx - sum(labelVec)) <= 1.e-10]:
                new_label_vec_list.append(labelVec + [newLabel])
        label_vec_list = new_label_vec_list

    new_label_vec_list = []
    for labelVec in label_vec_list:
        new_label_vec_list.append(labelVec + [m_approx - sum(labelVec)])
    label_vec_list = np.array(new_label_vec_list, int)

    return label_vec_list

def _validate_data(X_cand, return_utilities, batch_size,
                   random_state, reset=True, **check_X_cand_params):
    """Validate input data and set or check the `n_features_in_` attribute.
    Parameters
    ----------
    X_cand: array-like, shape (n_candidates, n_features)
        Candidate samples.
    batch_size : int,
        The number of samples to be selected in one AL cycle.
    return_utilities : bool,
        If true, also return the utilities based on the query strategy.
    random_state : numeric | np.random.RandomState, optional
        The random state to use.
    reset : bool, default=True
        Whether to reset the `n_features_in_` attribute.
        If False, the input will be checked for consistency with data
        provided when reset was last True.
    **check_X_cand_params : kwargs
        Parameters passed to :func:`sk.utils.check_array`.
    Returns
    -------
    X_cand: np.ndarray, shape (n_candidates, n_features)
        Checked candidate samples
    batch_size : int
        Checked number of samples to be selected in one AL cycle.
    return_utilities : bool,
        Checked boolean value of `return_utilities`.
    random_state : np.random.RandomState,
        Checked random state to use.
    """
    # Check candidate instances.
    X_cand = check_array(X_cand, **check_X_cand_params)

    # Check return_utilities.
    check_scalar(return_utilities, 'return_utilities', bool)

    # Check batch size.
    check_scalar(batch_size, target_type=int, name='batch_size',
                 min_val=1)
    batch_size = batch_size
    if len(X_cand) < batch_size:
        warnings.warn(
            "'batch_size={}' is larger than number of candidate samples "
            "in 'X_cand'. Instead, 'batch_size={}' was set ".format(
                batch_size, len(X_cand)))
        batch_size = len(X_cand)

    # Check random state.
    random_state = check_random_state(random_state=random_state,
                                      seed_multiplier=len(X_cand))

    return X_cand, return_utilities, batch_size, random_state

def check_scalar(x, name, target_type, min_inclusive=True, max_inclusive=True,
                 min_val=None, max_val=None):
    """Validate scalar parameters type and value.
    Parameters
    ----------
    x : object
        The scalar parameter to validate.
    name : str
        The name of the parameter to be printed in error messages.
    target_type : type or tuple
        Acceptable data types for the parameter.
    min_val : float or int, optional (default=None)
        The minimum valid value the parameter can take. If None (default) it
        is implied that the parameter does not have a lower bound.
    min_inclusive : bool, optional (default=True)
        If true, the minimum valid value is inclusive, otherwise exclusive.
    max_val : float or int, optional (default=None)
        The maximum valid value the parameter can take. If None (default) it
        is implied that the parameter does not have an upper bound.
    max_inclusive : bool, optional (default=True)
        If true, the maximum valid value is inclusive, otherwise exclusive.
    Raises
    -------
    TypeError
        If the parameter's type does not match the desired type.
    ValueError
        If the parameter's value violates the given bounds.
    """
    if not isinstance(x, target_type):
        raise TypeError('`{}` must be an instance of {}, not {}.'
                        .format(name, target_type, type(x)))
    if min_inclusive:
        if min_val is not None and x < min_val:
            raise ValueError('`{}`= {}, must be >= '
                             '{}.'.format(name, x, min_val))
    else:
        if min_val is not None and x <= min_val:
            raise ValueError('`{}`= {}, must be > '
                             '{}.'.format(name, x, min_val))

    if max_inclusive:
        if max_val is not None and x > max_val:
            raise ValueError('`{}`= {}, must be <= '
                             '{}.'.format(name, x, max_val))
    else:
        if max_val is not None and x >= max_val:
            raise ValueError('`{}`= {}, must be < '
                             '{}.'.format(name, x, max_val))
            
def check_random_state(random_state, seed_multiplier=None):
    """Check validity of the given random state.
    Parameters
    ----------
    random_state : None | int | instance of RandomState
        If random_state is None, return the RandomState singleton used by
        np.random.
        If random_state is an int, return a new RandomState.
        If random_state is already a RandomState instance, return it.
        Otherwise raise ValueError.
    seed_multiplier : None | int, optional (default=None)
        If the random_state and seed_multiplier are not None, draw a new int
        from the random state, multiply it with the multiplier, and use the
        product as the seed of a new random state.
    Returns
    -------
    random_state: instance of RandomState
        The validated random state.
    """
    if random_state is None or seed_multiplier is None:
        return sk.utils.check_random_state(random_state)

    check_scalar(seed_multiplier, name='seed_multiplier', target_type=int,
                 min_val=1)
    random_state = copy.deepcopy(random_state)
    random_state = sk.utils.check_random_state(random_state)

    seed = (random_state.randint(1, 2**31) * seed_multiplier) % (2**31)
    return np.random.RandomState(seed)

def _cost_reduction(k_vec_list, C=None, m_max=2, prior=1.e-3):
    """Calculate the expected cost reduction.
    Calculate the expected cost reduction for given maximum number of
    hypothetically acquired labels, observed labels and cost matrix.
    Parameters
    ----------
    k_vec_list: array-like, shape (n_samples, n_classes)
        Observed class labels.
    C: array-like, shape = (n_classes, n_classes)
        Cost matrix.
    m_max: int
        Maximal number of hypothetically acquired labels.
    prior : float | array-like, shape (n_classes)
       Prior value for each class.
    Returns
    -------
    expected_cost_reduction: array-like, shape (n_samples)
        Expected cost reduction for given parameters.
    """
    # Check if 'prior' is valid
    check_scalar(prior, 'prior', (float, int),
                 min_inclusive=False, min_val=0)

    # Check if 'm_max' is valid
    check_scalar(m_max, 'm_max', int, min_val=1)

    n_classes = len(k_vec_list[0])
    n_samples = len(k_vec_list)

    # check cost matrix
    C = 1 - np.eye(n_classes) if C is None else np.asarray(C)

    # generate labelling vectors for all possible m values
    l_vec_list = np.vstack([_gen_l_vec_list(m, n_classes)
                            for m in range(m_max + 1)])
    m_list = np.sum(l_vec_list, axis=1)
    n_l_vecs = len(l_vec_list)

    # compute optimal cost-sensitive decision for all combination of k-vectors
    # and l-vectors
    tile = np.tile(k_vec_list, (n_l_vecs, 1, 1))
    k_l_vec_list = np.swapaxes(tile, 0, 1) + l_vec_list
    y_hats = np.argmin(k_l_vec_list @ C, axis=2)

    # add prior to k-vectors
    prior = prior * np.ones(n_classes)
    k_vec_list = np.asarray(k_vec_list) + prior

    # all combination of k-, l-, and prediction indicator vectors
    combs = [k_vec_list, l_vec_list, np.eye(n_classes)]
    combs = np.asarray([list(elem)
                        for elem in list(itertools.product(*combs))])

    # three factors of the closed form solution
    factor_1 = 1 / euler_beta(k_vec_list)
    factor_2 = multinomial(l_vec_list)
    factor_3 = euler_beta(np.sum(combs, axis=1)).reshape(n_samples, n_l_vecs,
                                                         n_classes)

    # expected classification cost for each m
    m_sums = np.asarray(
        [factor_1[k_idx]
         * np.bincount(m_list, factor_2 * [C[:, y_hats[k_idx, l_idx]]
                                           @ factor_3[k_idx, l_idx]
                                           for l_idx in range(n_l_vecs)])
         for k_idx in range(n_samples)]
    )

    # compute classification cost reduction as difference
    gains = np.zeros((n_samples, m_max)) + m_sums[:, 0].reshape(-1, 1)
    gains -= m_sums[:, 1:]

    # normalize  cost reduction by number of hypothetical label acquisitions
    gains /= np.arange(1, m_max + 1)

    return np.max(gains, axis=1)

def simple_batch(
        utilities, random_state, batch_size=1, return_utilities=False):
    """Generates a batch by selecting the highest values in the 'utilities'.
    The returned utilities will be an 2D-array with the shape batch_size x
    len(utilities), filled the given utilities but set the n-th highest values
    in the n-th row to np.nan.
    Parameters
    ----------
    utilities : np.ndarray
        The utilities to be used to create the batch.
    random_state : numeric | np.random.RandomState
        The random state to use.
    batch_size : int, optional (default=1)
        The number of samples to be selected in one AL cycle.
    return_utilities : bool (default=False)
        If True, the utilities are returned.
    Returns
    -------
    best_indices : np.ndarray, shape (batch_size)
        The index of the batch instance.
    batch_utilities : np.ndarray,  shape (batch_size, len(utilities))
        The utilities of the batch (if return_utilities=True).
    """
    # validation
    utilities = check_array(utilities, ensure_2d=False, dtype=float)
    if batch_size == 'adaptive':
        batch_size = 1
    check_scalar(batch_size, target_type=int, name='batch_size',
                 min_val=1)
    if len(utilities) < batch_size:
        warnings.warn(
            "'batch_size={}' is larger than number of candidate samples "
            "in 'utilities'. Instead, 'batch_size={}' was set.".format(
                batch_size, len(utilities)))
        batch_size = len(utilities)
    # generate batch
    batch_utilities = np.empty((batch_size, len(utilities)))
    best_indices = np.empty(batch_size, dtype=int)
    for i in range(batch_size):
        best_indices[i] = rand_argmax(
            [utilities], axis=1, random_state=random_state)
        batch_utilities[i] = utilities
        utilities[best_indices[i]] = np.nan
    # Check whether utilities are to be returned.
    if return_utilities:
        return best_indices, batch_utilities
    else:
        return best_indices
    
# def predict_freq(classifier, X, classes):
#     """Return class frequency estimates for the input data X.
#     Parameters
#     ----------
#     X : array-like, shape (n_samples, n_features)
#         Input samples.
#     Returns
#     -------
#     F : array-like, shape (n_samples, classes)
#         The class frequency estimates of the input samples. Classes are
#         ordered according to classes_.
#     """
# #     check_is_fitted(classifier)
#     X = check_array(X)
#     classifier._check_n_features(X, reset=False)
    
#     # Counts number of votes per class label for each sample.
#     V = compute_vote_vectors(y=y, w=sample_weight,
#                              classes=np.arange(len(self.classes_)))

#     # Stores responsibility for every given sample of training set.
#     R = self.mixture_model_.predict_proba(X)

#     # Stores class frequency estimates per component.
#     self.F_components_ = R.T @ V
    
    
#     if np.sum(classifier.F_components_) > 0:
#         if classifier.weight_mode == 'similarities':
#             S = np.exp(-np.array(
#                 [cdist(X, [self.mixture_model_.means_[j]],
#                        metric='mahalanobis',
#                        VI=classifiermixture_model_.precisions_[j]).ravel()
#                  for j in range(classifier.mixture_model_.n_components)])).T
#         else:
#             S = classifier.mixture_model_.predict_proba(X)
#         F = S @ classifier.F_components_
#     else:
#         F = np.zeros((len(X), len(classifier.classes_)))
#     return F

In [21]:
# Method for calculating gain used by PAL sampling (before weighting by frequency estimates)
def pal_gain(K_c, m_max=2, alpha_c=1):
    """
    Calculates the expected performance gains given a maximal number of hypothetically acquired labels,
    the observed kernel frequency estimates and prior for the Dirichlet distribution.
    Parameters
    ----------
    K_c: array-like, shape (n_samples, n_classes)
        Observed kernel frequency estimates of the candidate samples.
    m_max: int, optional (default=2)
        Maximal number of hypothetically acquired labels.
    alpha_c : int | array-like, shape (n_classes), optional (default=1)
        Prior probabilities for the Dirichlet distribution.
        Default is 1 for all classes.
    Returns
    -------
    gains: array-like, shape (n_samples)
        Expected performance gains for given parameters.
    """
    n_classes = len(K_c[0])
    n_samples = len(K_c)

    # uniform risk matrix
    R = 1 - np.eye(n_classes)

    # generate labeling vectors for all possible m values
    l_vec_list = np.vstack([gen_l_vec_list(m, n_classes) for m in range(m_max + 1)])
    m_list = np.sum(l_vec_list, axis=1)
    n_l_vecs = len(l_vec_list)

    # compute optimal decision for all combination of k- and l-vectors
    k_l_vec_list = np.swapaxes(np.tile(K_c, (n_l_vecs, 1, 1)), 0, 1) + l_vec_list
    y_hats = np.argmin(k_l_vec_list @ R, axis=2)

    # add prior_classes to k-vectors
    alpha_c = alpha_c * np.ones(n_classes)
    K_c = np.asarray(K_c) + alpha_c

    # all combination of k-, l-, and prediction indicator vectors
    combs = [K_c, l_vec_list, np.eye(n_classes)]
    combs = np.asarray([list(elem) for elem in list(itertools.product(*combs))])

    # three factors of the closed form solution
    factor_1 = 1 / euler_beta(K_c)
    factor_2 = multinomial_coefficient(l_vec_list)
    factor_3 = euler_beta(np.sum(combs, axis=1)).reshape(n_samples, n_l_vecs, n_classes)

    # expected risk for each m
    m_sums = np.asarray(
        [factor_1[k_idx] * np.bincount(m_list, factor_2 * [R[:, y_hats[k_idx, l_idx]] @ factor_3[k_idx, l_idx]
                                                           for l_idx in range(n_l_vecs)]) for k_idx in
         range(n_samples)])

    # compute performance gains as risk reductions
    gains = np.zeros((n_samples, m_max)) + m_sums[:, 0].reshape(-1, 1)
    gains -= m_sums[:, 1:]

    # normalize performance gains by number of hypothetical label acquisitions
    gains /= np.arange(1, m_max + 1)

    return np.max(gains, axis=1)

# Method used by PAL for computing score for all candidate instances in pool
def compute_pal_scores(X_pool, n_classes, labeled_instances, labels, kernel, priors):
        """Compute score for each unlabeled sample. Score is to be maximized.
        Parameters
        ----------
        unlabeled_indices: array-like, shape (n_unlabeled_samples)
        Returns
        -------
        scores: array-like, shape (n_unlabeled_samples)
            Score of each unlabeled sample.
        """
        Z = np.eye(n_classes)[labels]

        K_x = kernel[:, :len(labeled_instances)] @ Z
        K_c = K_x[len(labeled_instances):]

        # calculate loss reduction for each unlabeled sample
        gains = pal_gain(K_c=K_c, alpha_c = priors)
        
        densities = np.sum(kernel, axis=1)
        gains *= densities[len(labeled_instances):]

        return gains


In [22]:
# Method for calculating gain used by xPAL sampling (before weighting by frequency estimates)
def xpal_gain(K_c, K_x=None, S=None, alpha_x=1, alpha_c=1):
    """
    Computes the expected probabilistic gain.
    Parameters
    ----------
    K_c: array-like, shape (n_candidate_samples, n_classes)
        Kernel frequency estimate vectors of the candidate samples.
    K_x: array-like, shape (n_evaluation_samples, n_classes), optional (default=K_c))
        Kernel frequency estimate vectors of the evaluation samples.
    S: array-like, shape (n_candidate_samples, n_evaluation_samples), optional (default=np.eye(n_candidate_samples))
        Similarities between all pairs of candidate and evaluation samples
    alpha_x: array-like, shape (n_classes)
        Prior probabilities for the Dirichlet distribution of the samples in the evaluation set.
        Default is 1 for all classes.
    alpha_c: float | array-like, shape (n_classes)
        Prior probabilities for the Dirichlet distribution of the candidate samples.
        Default is 1 for all classes.
    Returns
    -------
    gains: numpy.ndarray, shape (n_candidate_samples)
        Computed expected gain for each candidate sample.
    """
    # check kernel frequency estimates of candidate samples
    K_c = check_array(K_c)
    n_candidate_samples = K_c.shape[0]
    n_classes = K_c.shape[1]

    # check kernel frequency estimates of evaluation samples
    K_x = K_c if K_x is None else check_array(K_x)
    n_evaluation_samples = K_x.shape[0]
    if n_classes != K_x.shape[1]:
        raise ValueError("'K_x' and 'K_c' must have one column per class")

    # check similarity matrix
    S = np.eye(n_candidate_samples) if S is None else check_array(S)
    if S.shape[0] != n_candidate_samples or S.shape[1] != n_evaluation_samples:
        raise ValueError("'S' must have the shape (n_candidate_samples, n_evaluation_samples)")

    # check prior parameters
    if hasattr(alpha_c, "__len__") and len(alpha_c) != n_classes:
        raise ValueError("'alpha_c' must be either a float > 0 or array-like with shape (n_classes)")
    if hasattr(alpha_x, "__len__") and len(alpha_x) != n_classes:
        raise ValueError("'alpha_x' must be either a float > 0 or array-like with shape (n_classes)")

    # uniform risk matrix
    R = 1 - np.eye(n_classes)

    # model future hypothetical labels
    l_vecs = np.eye(n_classes, dtype=int)

    # compute possible risk differences
    class_vector = np.arange(n_classes, dtype=int)
    R_diff = np.array([[R[:, y_hat] - R[:, y_hat_l] for y_hat_l in class_vector] for y_hat in class_vector])

    # compute current error per evaluation sample and class
    R_x = K_x @ R

    # compute current predictions
    y_hat = np.argmin(R_x, axis=1)

    # compute required labels per class to flip decision
    with np.errstate(divide='ignore', invalid='ignore'):
        D_x = np.nanmin(np.divide(R_x - np.min(R_x, axis=1, keepdims=True), R[:, y_hat].T), axis=1)
        D_x = np.tile(D_x, (len(S), 1))

    # indicates where a decision flip can be reached
    I = D_x - S < 0
    # print('#decision_flips: {}'.format(np.sum(I)))

    # compute normalization constants per candidate sample
    K_c_alpha_c_norm = K_c + alpha_c
    K_c_alpha_c_norm /= K_c_alpha_c_norm.sum(axis=1, keepdims=True)

    # stores gain per candidate sample
    gains = np.zeros(n_candidate_samples)

    target_weight = 100
    non_target_weight = 0.05
    
    # compute gain for each candidate sample
    flip_indices = np.argwhere(np.sum(I, axis=1) > 0)[:, 0]
    for ik_c in flip_indices:
        for l_idx, l_vec in enumerate(l_vecs):
            if l_vec[0] == 0:
                weight = non_target_weight
            else:
                weight = target_weight
            
            K_l = (S[ik_c, I[ik_c]] * l_vec[:, np.newaxis]).T
            K_new = K_x[I[ik_c]] + K_l
            y_hat_l = np.argmin(K_new @ R, axis=1)
            K_new += alpha_x
            K_new /= np.sum(K_new, axis=1, keepdims=True)
            gains[ik_c] += K_c_alpha_c_norm[ik_c, l_idx] * np.sum(K_new * R_diff[y_hat[I[ik_c]], y_hat_l])
            
    gains /= n_evaluation_samples
    
    return gains

# Method used by xPAL for computing score for all candidate instances in pool
def compute_xpal_scores(X_pool, n_classes, labeled_instances, labels, kernel, priors):
    """Compute score for each unlabeled sample. Score is to be maximized.
    Parameters
    ----------
    unlabeled_indices: array-like, shape (n_unlabeled_samples)
    Returns
    -------
    scores: array-like, shape (n_unlabeled_samples)
        Score of each unlabeled sample.
    """
    # compute frequency estimates for evaluation set (K_x) and candidate set (K_c)
    Z = np.eye(n_classes)[labels]

    K_x = kernel[:, :len(labeled_instances)] @ Z
    K_c = K_x[len(labeled_instances):]

    # calculate loss reduction for each unlabeled sample
    gains = xpal_gain(K_c=K_c, K_x = K_x, S = kernel[len(labeled_instances):], alpha_x = priors, alpha_c = priors) # S = kernel[len(labeled_instances)] or S= kernel

    return gains

In [ ]:
run_openml_test(datak_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

In [23]:
# Setting up a PAL sampling method given a classifier and a candidate pool of instances to be sampled
def pal_sampling(classifier, X_pool, n_classes, labeled_instances, labels, kernel, priors):
    scores = compute_pal_scores(X_pool, n_classes, labeled_instances, labels, kernel, priors)
    query_idx = shuffled_argmax(scores, n_instances=1)
    return query_idx, X_pool[query_idx]
    

In [24]:
# Setting up a xPAL sampling method given a classifier and a candidate pool of instances to be sampled
def xpal_sampling(classifier, X_pool, n_classes, labeled_instances, labels, kernel, priors):
    scores = compute_xpal_scores(X_pool, n_classes, labeled_instances, labels, kernel, priors)
    query_idx = shuffled_argmax(scores, n_instances=1)
    return query_idx, X_pool[query_idx]

In [27]:
# Switcher for selecting the desired classifier
ML_switcher = {
        1: LogisticRegression(solver='liblinear', n_jobs = -1),
        2: RandomForestClassifier(n_jobs = -1),
        3: xgboost.XGBClassifier(booster="dart", eval_metric = 'error', use_label_encoder=False, n_jobs = -1),
        4: DecisionTreeClassifier(random_state=None),
        5: SVC(kernel='linear', probability=True)
}

# Switcher for selecting the desired AL method
AL_switcher = {
        1: random_sampling,
        2: uncertainty_sampling,
        3: density_sampling,
        4: expected_error_reduction,
        5: qbc_sampling,
        6: pal_sampling,
        7: xpal_sampling
}

In [28]:
# The base definitions of all test settings. These are set with new values when calling the run_test method (or set_settings method).

# Ideally will have at least 50 repetitions of validation of one method to get a good idea of performance. repetitions = K * EXECUTIONS
K = 5 # number of folds for validation
EXECUTIONS = 10 # number of executions of k-fold validation

# Number of queries per round of learning. The main testing loops in run_test could be adapted to not be static. E.g. until convergence, or some other stopping criterion. 
# However, for evaluation it is easier if all methods are tested with the same number of queries (requirement of Area Under the Learning Curve measure (ALC)).
# Performance is evaluated after each instance is queried and added to the training pool.
N_QUERIES = 45

# size of initialization set. Needs to be at least the # classes, but more if QBC used.
INITIALIZATION_SET = 10

# Whether to train/evaluate only on the summarized trajectories, ele all instances of each trajectory are used for training.
TRAIN_ON_SUM = True

# Whether separte classifiers should be used for querying and classifying (producer/consumer respectively)
# *note, for QBC should petty much always use separate model. Really doesn't make sense not to.
SEPARATE_CLASSIFIER = False

# Classifier to be used:
# 1 = logistic regression, 2 = Random Forest, 3 = XGBoost, 4 = Decision Tree, 5 = SVM (linear version)
ML_METHOD = 3
ML_SEP_METHOD = 3 # only relevant if using separate classifier as query producer. Overridden when using QBC

# Active learning approach to be used:
# 1 = random sampling, 2 = uncertainty sampling, 3 = density-weighted sampling, 4 = EER, 5 = QBC, 6 = PAL, 7 = XPAL
AL_METHOD = 6

# Specific learners used for the committee in QBC
QBC_LEARNERS = [2,3,5]

# The number of learners of each classifier type specified by QBC_LEARNERS to be used in QBC
N_QBC_LEARNERS = 3

# Selection of QBC disagreement measure
# vote_entropy_sampling, consensus_entropy_sampling, max_disagreement_sampling
QBC_STRATEGY = max_disagreement_sampling

# Set whether to oversample, undersample, or neither. Should NOT have over- and undersampling both be True
OVERSAMPLE = False
UNDERSAMPLE = False
SAMPLING_RATIO = 0.25

# Weighing factor, (in favor of the targete class) 
WEIGHTING_FACTOR = 0.15

# Whether to scale data and apply PCA (PCA currently disabled -> commented out)
SC_PCA = False

# whether to save results from current test as figures
SAVEFIGS = False
SAVERATIO = False
SAVERESULTS = True

In [20]:
# Method for altering settings between runs. Settings that should generally stay consistent between runs are not changed by this method. 
# This can also be done by the main run_test method, making this method obsolete.
def set_settings(k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, bandwidth_ = 0, sc_pca_ = True):
    global K, EXECUTIONS, N_QUERIES, INITIALIZATION_SET, TRAIN_ON_SUM, SEPARATE_CLASSIFIER, ML_METHOD, ML_SEP_METHOD, AL_METHOD, QBC_LEARNERS, N_QBC_LEARNERS, OVERSAMPLE, UNDERSAMPLE, WEIGHTING_FACTOR, SAMPLING_RATIO, SAVEFIGS, SAVERATIO, SAVERESULTS, BANDWIDTH, SC_PCA
    K = k_
    EXECUTIONS = execs_
    N_QUERIES = n_queries_
    INITIALIZATION_SET = n_init_
    TRAIN_ON_SUM = train_sum_
    
    SEPARATE_CLASSIFIER = sep_class_
    ML_METHOD = ml_method_
    ML_SEP_METHOD = ml_sep_method_
    
    AL_METHOD = al_method_
    QBC_LEARNERS = qbc_learners_
    N_QBC_LEARNERS = n_qbc_learners_
    
    # normally calculated if set to 0
    BANDWIDTH = bandwidth_
    
    # Undersampling or oversampling according to the sampling ratio defined in the base settings
    UNDERSAMPLE = False
    OVERSAMPLE = False

    WEIGHTING_FACTOR = weighting_factor_
    SC_PCA = sc_pca_
    
    SAVEFIGS = save_figs_
    SAVERATIO = save_ratio_
    SAVERESULTS = save_results_

In [21]:
def run_openml_test(data, k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, time_point = None, bandwidth_ = 0, pal_priors = 1, sc_pca_ = True):
    # Seed to allow different AL methods to train and test with the same splits and initialization sets over the course of the same number of executions with the same settings.
    np.random.seed(0)
    # Call the set_settings method to change settings as defined by the parameters of this method.
    set_settings(k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, bandwidth_, sc_pca_)
    
    # Keep track of results for each query. Used later to calculate incremental performance. +1 because the first value is the performance directly after initialization
    accuracy_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    macro_f1_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    recall_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    precision_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    auc_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    
    # Keep track of feature importance during training. 
    feat_importance_cons = pd.DataFrame([], columns = range(N_QUERIES + 1), index = range(EXECUTIONS * K))
    if SEPARATE_CLASSIFIER:
        feat_importance_prod = pd.DataFrame([], columns = range(N_QUERIES + 1), index = range(EXECUTIONS * K))

    # Keep track of label ratio in data and selected label ratio
    balance_ratios = pd.DataFrame(columns = ["Training Folds Ratio", "Selected Training Instances Ratio"])
    selected_instances_table = pd.DataFrame(columns = range(N_QUERIES))

    # Keep track of selection frequencies of individual trajectories
    #trajectory_selection_frequencies = pd.DataFrame(0, index=range(N_QUERIES), columns = looping_sum[trajectory])

    # Compute weighting factors for when instance weighting is selected.
    #non_target_weight = np.unique(compute_sample_weight('balanced', looping_sum[target]), return_counts=True)[0][0] * (2-WEIGHTING_FACTOR)
    #target_weight = np.unique(compute_sample_weight('balanced', looping_sum[target]), return_counts=True)[0][1] * WEIGHTING_FACTOR
    
    # Perform K-fold validation for EXECUTIONS number of times. This gives more total repetitions, and thus a smoother view of learning performance
    for execs in range(EXECUTIONS):
        df = data

        print("Starting execution: ", execs+1, "/" , EXECUTIONS,  "- - - - - - - - ")

        unique_classes = df[target].unique()
        n_labeled_examples = df.shape[0]

        if len(unique_classes) > INITIALIZATION_SET:
            print("INCREASE SIZE OF INITIALIZATION SET TO INCLUDE AT LEAST ONE INSTANCE OF EACH CLASS")
            raise
        
        # Make random selection for initialization set based on the sampling ratio and initialization set size
        n_minority = math.ceil(int(INITIALIZATION_SET * SAMPLING_RATIO))
        n_majority = INITIALIZATION_SET - n_minority
        
        initialization_indices = np.hstack([df[df[target]==NUM_TARGET].sample(n=n_minority, replace=False).index, \
                                                 df[df[target]!=NUM_TARGET].sample(n=n_majority, replace=False).index])
        initialization_set = df.iloc[initialization_indices]

        # Isolate the non-initialization examples to make up the sampling pool.
        df = df.drop(df.index[initialization_indices]).reset_index(drop=True)

        # Create separate set of initialization indices if using full trajectories for training
        if not TRAIN_ON_SUM:
            full_initialization_set = pd.DataFrame(columns = full_df.columns)

            for unique_trajectory in initialization_set[trajectory].unique():
                # Retrieve all instances with this trajectory id from full data set
                full_initialization_set = full_initialization_set.append(full_df[full_df[trajectory] == unique_trajectory])

            # Split into X and y
            X_initialize = full_initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = full_initialization_set[target].reset_index(drop=True)
            del(full_initialization_set)
        else:
            # Labeled training instances.
            X_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = initialization_set[target].reset_index(drop=True)
            
        # labeled training instances.
        X_aggr_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
        y_aggr_initialize = initialization_set[target].reset_index(drop=True)

        X_initialize[trajectory] = X_initialize[trajectory].apply(pd.to_numeric)
        y_initialize = pd.to_numeric(y_initialize)

        # Take stratified folds to make sure each fold contains enough instances of the majority class
        skf = StratifiedKFold(n_splits=K, shuffle = True)
        
        if time_point != None:
            pretime_df, ontime_df = time_specific_summarized_set(df, time_point, 30)
    
            X_aggr = ontime_df.drop(target, axis=1).to_numpy()
            y_aggr = ontime_df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))
            
        else:
            # Split aggregated instances into X and y
            X_aggr = df.drop(target, axis=1).to_numpy()
            y_aggr = df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))

        # K-fold cross validation
        for i in range(K):
            print("Validating on fold: ", i+1 , "/", K, end="\r")
            # Set train and test set for current fold
            if time_point != None:
                X_train = np.vstack([X_aggr[train_set_indices[i][0]], pretime_df.drop(target, axis=1).to_numpy()])
                y_train = np.hstack([y_aggr[train_set_indices[i][0]], pretime_df[target].to_numpy()])
        
                X_test = X_aggr[train_set_indices[i][1]]
                y_test = y_aggr[train_set_indices[i][1]]
        
            else: 
                X_train = X_aggr[train_set_indices[i][0]]
                y_train = y_aggr[train_set_indices[i][0]]
                
                X_test = X_aggr[train_set_indices[i][1]]
                y_test = y_aggr[train_set_indices[i][1]]
            
            # If the training set has fewer instances than the selected query budget, we have too few instances
            if len(X_train) < N_QUERIES:
                print("TOO FEW TRAINING INSTANCES TO TRAIN THIS FAR. REDUCE TO ", len(X_train))
                raise
            
            # This is why the trajectory identifier needs to be the first column. For creating the folds the dataframe needs to be turned into a numpy array,
            # but because of this the ability to select based on name is lost when we try to find these identifiers later.
            unique_train_trajectories = np.unique(X_train[:,0])
            unique_test_trajectories = np.unique(X_test[:,0])
            unique_initialization_trajectories = X_initialize[trajectory].unique()
            
            fold_full_df = full_df[full_df[trajectory].isin(np.concatenate([unique_initialization_trajectories, unique_train_trajectories, unique_test_trajectories]))]
            
            # Line only needed when testing without PCA and scaling, but doesn't do harm when not using PCA or scaling
            fold_X_initialize = X_initialize

            # Currently PCA commented out due to performance decrease for testing during the project. Can be uncommented to inlude PCA again
            if SC_PCA:
                # Initialization of SC and PCA:
                sc = StandardScaler()
                pca = PCA(n_components=0.9)

                # If training with all trajectory instance, need to scale and fit for this separately from the summarized dataset
                if not TRAIN_ON_SUM:
                    sc.fit(fold_full_df[fold_full_df[trajectory].isin(unique_train_trajectories)].drop(trajectory,axis=1).drop(target,axis=1))
                    fold_full_df = fold_full_df[[trajectory, target]].join(pd.DataFrame(sc.transform(fold_full_df.drop(trajectory,axis=1).drop(target,axis=1)), \
                                                                                            columns=fold_full_df.columns[2:], index=fold_full_df.index))
#                     pca.fit(fold_full_df[fold_full_df[trajectory].isin(unique_train_trajectories)].drop(trajectory,axis=1).drop(target,axis=1))
#                     fold_full_df = fold_full_df[[trajectory, target]].join(pd.DataFrame(pca.transform(fold_full_df.drop(trajectory,axis=1).drop(target,axis=1)), \
#                                                                                             columns=['PCA%i' %  i for i in range(len(pca.components_))], index=fold_full_df.index))

                    # if not using a separte classifier, need to apply same scaling and PCA of full dataset to the summarized one so training and predictions sets don't have different features
                    if SEPARATE_CLASSIFIER:
                        X_train[:,1:] = sc.transform(X_train[:,1:])
                        X_test[:,1:] = sc.transform(X_test[:,1:])
                        X_initialize_values = X_initialize.drop(trajectory,axis=1)
                        X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                        fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                         X_train = np.hstack((X_train[:,0,np.newaxis], pca.transform(X_train[:,1:])))
#                         X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                         X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                         X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                         fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
                    else:
                        X_train[:,1:] = sc.fit_transform(X_train[:,1:])
                        X_test[:,1:] = sc.transform(X_test[:,1:])
                        X_initialize_values = X_initialize.drop(trajectory,axis=1)
                        X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                        fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                         X_train = np.hstack((X_train[:,0,np.newaxis], pca.fit_transform(X_train[:,1:])))
#                         X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                         X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                         X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                         fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
                else:
                    X_train[:,1:] = sc.fit_transform(X_train[:,1:])
                    X_test[:,1:] = sc.transform(X_test[:,1:])
                    X_initialize_values = X_initialize.drop(trajectory,axis=1)
                    X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                    fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                     X_train = np.hstack((X_train[:,0,np.newaxis], pca.fit_transform(X_train[:,1:])))
#                     X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                     X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                     X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                     fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
            
            # Backup that allows for multiple ways of using the same y_test name but handling performance measures in different ways
            y_test_original = y_test

            if OVERSAMPLE and UNDERSAMPLE:
                print("CAN'T UNDER AND OVERSAMPLE AT THE SAME TIME")
                raise
            if OVERSAMPLE:
                sampler = RandomOverSampler(sampling_strategy=SAMPLING_RATIO)
                X_train, y_train = sampler.fit_resample(X_train, y_train)
            elif UNDERSAMPLE:
                sampler = RandomUnderSampler(sampling_strategy=SAMPLING_RATIO)
                X_train, y_train = sampler.fit_resample(X_train, y_train)

            # Save class ratio of instances in training fold
            fold_training_ratio = np.count_nonzero(y_train == NUM_TARGET)/np.count_nonzero(y_train != NUM_TARGET) # MOSTLY MAKES SENSE FOR BINARY TARGET CLASS

            model = ML_switcher.get(ML_METHOD)

            if AL_METHOD == 5 and not SEPARATE_CLASSIFIER: # Using QBC for main classifier, NOT RECOMMENDED WHEN COMPARING AL METHODS
                committee_list = []
                unique_trajectories_init = fold_X_initialize[trajectory].unique()
                qbc_initialization_size = math.ceil(INITIALIZATION_SET/(N_QBC_LEARNERS*len(QBC_LEARNERS))) + 1
                
                # For each specified QBC member, create N_QBC_LEARNERS learners to be added to the committee
                for qbc_model in QBC_LEARNERS:
                    # select random numbers, find trip id's for these in the aggr dataset. Then select all instances with these id's from the initilizing set and use them as new initializing set
                    for n in range(N_QBC_LEARNERS):
                        original_estimator = ML_switcher.get(qbc_model)
                        new_estimator = deepcopy(original_estimator)
                        
                        # The number of items for initializing the committe members is the proportion of number of members to size of the initialization set, rounded up.
                        # Considering the initialization set is usually small in this case due to low amounts of data, replacement is allowed which results insome overlap 
                        # in intialization sets make random selection for initialization set based on the sampling ratio and initialization set size
                        n_minority = math.ceil(int(qbc_initialization_size * SAMPLING_RATIO)) + 1
                        n_majority = qbc_initialization_size - n_minority

                        unique_negative_target = np.random.choice(initialization_set[initialization_set[target] != NUM_TARGET][trajectory].unique(),n_majority,replace=True)
                        unique_positive_target = np.random.choice(initialization_set[initialization_set[target] == NUM_TARGET][trajectory].unique(),n_minority,replace=True)
                        trajectories_init = np.concatenate((unique_positive_target, unique_negative_target), axis=None)
                        
                        idx_member_init = fold_X_initialize.index[fold_X_initialize[trajectory].isin(trajectories_init)].to_numpy()

                        X_member_init = fold_X_initialize.iloc[idx_member_init].to_numpy()
                        y_member_init = y_initialize.iloc[idx_member_init].to_numpy()

                        # Initializing learner
                        committee_member =  ActiveLearner(
                            estimator=new_estimator,
                            X_training=X_member_init, y_training=y_member_init
                        )
                        committee_list.append(committee_member)

                # Assembling the committee
                learner = Committee(
                    learner_list=committee_list,
                    query_strategy=QBC_STRATEGY
                )
            else: # NOT using QBC for main classifier. Should pretty much be always the case
                # Specify the core estimator along with its active learning model. Fit with the labeled data set
                learner = ActiveLearner(
                    estimator=deepcopy(model),
                    query_strategy=AL_switcher.get(AL_METHOD),
                    X_training=fold_X_initialize.to_numpy(), y_training=y_initialize.to_numpy()
                )

            # Create separate classifer for training and estimation
            if SEPARATE_CLASSIFIER:
                query_model = deepcopy(ML_switcher.get(ML_SEP_METHOD))
                new_query_strategy = deepcopy(AL_switcher.get(AL_METHOD))

                if AL_METHOD != 5:
                    query_estimator = ActiveLearner(
                            estimator=query_model,
                            query_strategy=new_query_strategy,
                            X_training=X_aggr_initialize.to_numpy(), y_training=y_aggr_initialize.to_numpy()
                    )
                else:
                    committee_list = []
                    unique_trajectories_init = X_aggr_initialize[trajectory].unique()
                    qbc_initialization_size = math.ceil(INITIALIZATION_SET/(N_QBC_LEARNERS*len(QBC_LEARNERS)))+1
                    
                    # For each specified QBC member, create N_QBC_LEARNERS learners to be added to the committee
                    for qbc_model in QBC_LEARNERS:
                        
                        # select random numbers, find trip id's for these in the aggr dataset. Then select all instances with these id's from the initilizing set and use them as new initializing set
                        for n in range(N_QBC_LEARNERS):
                            original_estimator = deepcopy(ML_switcher.get(qbc_model))
                            new_estimator = deepcopy(original_estimator)
                            
                            # make random selection for initialization set based on the sampling ratio and initialization set size
                            n_minority = math.ceil(int(qbc_initialization_size * SAMPLING_RATIO)) + 1
                            n_majority = qbc_initialization_size - n_minority

                            unique_negative_target = np.random.choice(initialization_set[initialization_set[target] != NUM_TARGET][trajectory].unique(),n_majority,replace=False)
                            unique_positive_target = np.random.choice(initialization_set[initialization_set[target] == NUM_TARGET][trajectory].unique(),n_minority,replace=False)
                            
                            trajectories_init = np.hstack([unique_positive_target, unique_negative_target])

                            idx_member_init = X_aggr_initialize[X_aggr_initialize[trajectory].isin(trajectories_init)].index.to_numpy()

                            X_member_init = X_aggr_initialize.iloc[idx_member_init].to_numpy()
                            y_member_init = y_aggr_initialize.iloc[idx_member_init].to_numpy()

                            # initializing learner
                            committee_member =  ActiveLearner(
                                estimator=new_estimator,
                                X_training=X_member_init, y_training=y_member_init
                            )
                            committee_list.append(committee_member)

                    # Assembling the committee
                    query_estimator = Committee(
                        learner_list=committee_list,
                        query_strategy=vote_entropy_sampling
                    )

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
            # Make first set of predictions (before querying)
            predictions = []

            if TRAIN_ON_SUM:
                predictions = learner.predict(X_test)

            else:
                # Training with all instances performance 1: Evaluating performance only for the summarizing instance (in this case single point at end of trajectory)
                predictions_single = learner.predict(X_test)

                unqueried_score_single = accuracy_score(y_test, predictions_single)
                unqueried_macro_f1_single = f1_score(y_test, predictions_single, average='macro')
                unqueried_recall_single = recall_score(y_test, predictions_single, average='macro')
                unqueried_precision_single = precision_score(y_test, predictions_single, average='macro', zero_division=0)
                unqueried_auc_single = roc_auc_score(y_test, predictions_single, average = 'macro')

                # Store performance of every calculated score
                accuracy_history_single = [unqueried_score_single]
                macro_f1_history_single = [unqueried_macro_f1_single]
                recall_history_single = [unqueried_recall_single]
                precision_history_single = [unqueried_precision_single]
                auc_history_single = [unqueried_auc_single]

                # Training with all instances performance 2: Evaluate performance by classifying all individual instances of each trajectory and taking a majority vote to get a 
                # prediction for that one trajectory.
                y_test = []
                for unique_trajectory in unique_test_trajectories:
                    # Retrieve all instances with this trajectory id from full data set
                    test_trajectory_full = fold_full_df[fold_full_df[trajectory] == unique_trajectory]
                    # Split into numpy X and y
                    X_test_full = test_trajectory_full.drop(target, axis=1).to_numpy()

                    y_test.append(test_trajectory_full[target].to_numpy()[0])
                    del(test_trajectory_full)

                    # get prediction of one trajectory via majority vote of all its inividual instance classifications
                    trajectory_prediction = np.bincount(learner.predict(X_test_full).astype(int)).argmax() # not sure why this has to be cast as int, already should be

                    # append prediction to a list
                    predictions.append(trajectory_prediction)
                del(X_test_full)

            # Record our learner's performance on the test data
            unqueried_score = accuracy_score(y_test, predictions)
            unqueried_macro_f1 = f1_score(y_test, predictions, average='macro')
            unqueried_recall = recall_score(y_test, predictions, average='macro')
            unqueried_precision = precision_score(y_test, predictions, average='macro', zero_division=0)
            unqueried_auc = roc_auc_score(y_test, predictions, average = 'macro')

            # Store performance of every calculated score
            accuracy_history = [unqueried_score]
            macro_f1_history = [unqueried_macro_f1]
            recall_history = [unqueried_recall]
            precision_history = [unqueried_precision]
            auc_history = [unqueried_auc]

            # Store feature importance.  For QBC, average importances of all committe members taken (although the differences between the members could be interesting)
            if AL_METHOD != 5:
                if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                    feat_importance_cons.iat[0,0] = abs(learner.estimator.coef_)[0]
                else:
                    feat_importance_cons.iat[0,0] = learner.estimator.feature_importances_
                if SEPARATE_CLASSIFIER:
                    if query_estimator.estimator.__class__.__name__ == "SVC" or query_estimator.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_prod.iat[0,0] = abs(query_estimator.estimator.coef_)[0]
                    else:
                        feat_importance_prod.iat[0,0] = query_estimator.estimator.feature_importances_
            else:
                if not SEPARATE_CLASSIFIER:    
                    feat_importance_cons.iat[0,0] = 0
                    for member in learner:
                        if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_cons.iat[0,0] += abs(member.estimator.coef_)[0]
                        else:
                            feat_importance_cons.iat[0,0] += member.estimator.feature_importances_
                        feat_importance_cons.iat[0,0] = feat_importance_cons.iat[0,0] / len(learner)
                else:
                    if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_cons.iat[0,0] = abs(learner.estimator.coef_)[0]
                    else:
                        feat_importance_cons.iat[0,0] = learner.estimator.feature_importances_

                    feat_importance_prod.iat[0,0] = 0
                    for member in query_estimator:
                        if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_prod.iat[0,0] += abs(member.estimator.coef_)[0]
                        else:
                            feat_importance_prod.iat[0,0] += member.estimator.feature_importances_
                        feat_importance_prod.iat[0,0] = feat_importance_prod.iat[0,0] / len(query_estimator)
            
            # Labels of selected training instances. Used for calculating metrics later on
            selected_training_instance_labels = np.array([])
            
            
            # For PAL: calculating gamma through bandwidth, with mean as default and using rbf as the kernel
            nominator = 2 * len(X_test) * len(X_test[0]) # number of instances, number of features
            denominator = (len(X_test) - 1) * np.log((len(X_test) - 1) / ((np.sqrt(2) * 10 ** -6) ** 2))
            bandwidth = np.sqrt(nominator / denominator)
            
            if BANDWIDTH == 0:
                bandwidth = np.sqrt(nominator / denominator)
            else: bandwidth = BANDWIDTH
            
            gamma = 0.5 * (bandwidth ** (-2))
            
            # Perform training and testing for the budget of N_QUERIES. The "unlabeled" dataset is queried for the most informative points according to our query strategy
            # which are consecutively added to the training set after which performance is directly measured.
            for index in range(N_QUERIES):
                # Setting up kernel with rbf as default and additional parameters (for PAL and xPAL)
                params = {}
                if AL_METHOD == 6 or AL_METHOD == 7 or AL_METHOD == 8 or AL_METHOD == 9:
                    S =  pairwise_kernels(X = X_train, Y = X_train, metric='rbf', gamma=gamma)
                    S = check_array(S, np.eye(len(X_train)))
                    if SEPARATE_CLASSIFIER:
                        params = {'n_classes': len(unique_classes), "labeled_instances": query_estimator.X_training, "labels": query_estimator.y_training, "kernel": S, "priors": pal_priors}
                    else: 
                        params = {'n_classes': len(unique_classes), "labeled_instances": learner.X_training, "labels": learner.y_training, "kernel": S, "priors": pal_priors}

                # Query instance using AL method
                if SEPARATE_CLASSIFIER:
                    query_index, query_instance = query_estimator.query(X_train, **params)
                else:
                    query_index, query_instance = learner.query(X_train, **params)

                # For measuring frequencies of selection of specific trajectories over the course of learning
                trajectory_selection_frequencies.iloc[index][int(X_train[query_index,0])] += 1
                
                # For measuring performance in terms of label selection
                selected_training_instance_labels = np.append(selected_training_instance_labels, y_train[query_index])

                # logic to make sure only do additional bootstrapping when using qbc, and only on the QBC classifiers
                boot_main = False
                boot_sep = False
#                 # This currently breaks. The bootstrapping takes random samples, but there are too few in the initialization set to guarantee at least one of each class
#                 if AL_METHOD == 5:
#                     if SEPARATE_CLASSIFIER:
#                         boot_main = False
#                         boot_sep = True
#                     else:
#                         boot_main = True
#                         boot_sep = True
                
                # Setting weighting of training instances when not randomly selecting them. Using two arrays guarantees that no overlapping values erase each other
                #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                #    current_labels = np.append(learner.y_training, y_train[query_index].reshape(1, ))
                #    sample_weights = np.zeros(len(current_labels)).astype(float)
                #    temp_indices = np.where(current_labels == NUM_TARGET)
                #    sample_weights[temp_indices] = target_weight
                #    temp_indices = np.where(current_labels != NUM_TARGET)
                #    sample_weights[temp_indices] = non_target_weight
                #else:
                #    sample_weights = None
    
        
                # Training the consumer with the summarizing queried instance
                if TRAIN_ON_SUM:
                    # Train our ActiveLearner model with the instance it has requested.
                    X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                    learner.teach(X=X_temp, y=y_temp, bootstrap=boot_main, sample_weight = sample_weights)

                    # Training a separate classifier (producer) with the summarizing queried instance
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap=boot_sep, sample_weight = sample_weights)

                # Training the model with all instances of queried trip
                else:
                    # Training a separate classifier (producer) used for selecting queries
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        
                        #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                        #    current_labels = np.append(query_estimator.y_training, y_train[query_index].reshape(1, ))
                        #    sample_weights = np.zeros(len(current_labels)).astype(float)
                        #    temp_indices = np.where(current_labels == NUM_TARGET)
                        #    sample_weights[temp_indices] = target_weight
                        #    temp_indices = np.where(current_labels != NUM_TARGET)
                        #    sample_weights[temp_indices] = non_target_weight
                        #else:
                        #    sample_weights = None
                        
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap = boot_sep, sample_weight = sample_weights)

                    # Find trip_id of selected query in aggregated instances
                    query_trajectory_id = X_train[query_index,0]
                    # Retrieve all instances with this trajectory id from full data set

                    if not isinstance(query_trajectory_id, float):
                        query_trajectory_id = query_trajectory_id[0]
                    query_trajectory_full = fold_full_df[fold_full_df[trajectory] == query_trajectory_id] # for sep classifier the specific formatting returns one element in two dimensions

                    # Split into numpy X and y
                    X_train_full = query_trajectory_full.drop(target, axis=1).to_numpy()
                    y_train_full = query_trajectory_full[target].to_numpy()
                    del(query_trajectory_full)

                    #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                    #    current_labels = np.append(learner.y_training, y_train_full)
                    #    sample_weights = np.zeros(len(current_labels)).astype(float)
                    #    temp_indices = np.where(current_labels == NUM_TARGET)
                    #    sample_weights[temp_indices] = target_weight
                    #    temp_indices = np.where(current_labels != NUM_TARGET)
                    #    sample_weights[temp_indices] = 1 - (non_target_weight * 0.1)
                    #else:
                    #    sample_weights = None
                    
                    # Add selected instances to training pool and teach the model re-train the model on the new set
                    # Train the consumer on either the full or 
                    learner.teach(X=X_train_full, y=y_train_full, bootstrap=boot_main, sample_weight = sample_weights)

                    del(X_train_full)
                    del(y_train_full)

                # Remove the queried instance from the unlabeled pool.
                X_train, y_train = np.delete(X_train, query_index, axis=0), np.delete(y_train, query_index)

                predictions = []
                # Evluate performance of
                if TRAIN_ON_SUM:
                    # Make predictions
                    predictions = learner.predict(X_test)

                # Evaluate performance on full trajectories by classifying all individual instances of each trajectory and taking a majority vote to get a prediction for that one trajectory
                else:
                    predictions_single = learner.predict(X_test)

                    # Calculate and report our model's accuracy for last instance.
                    model_accuracy_single = accuracy_score(y_test_original, predictions_single)
                    model_macro_f1_single = f1_score(y_test_original, predictions_single, average='macro')
                    model_recall_single = recall_score(y_test_original, predictions_single, average='macro')
                    model_precision_single = precision_score(y_test_original, predictions_single, average='macro', zero_division=0) # zero_division because sometimes no preictions of looping class
                    model_auc_single = roc_auc_score(y_test_original, predictions_single, average = 'macro')
                    
                    accuracy_history_single.append(model_accuracy_single)
                    macro_f1_history_single.append(model_macro_f1_single)
                    recall_history_single.append(model_recall_single)
                    precision_history_single.append(model_precision_single)  
                    auc_history_single.append(model_auc_single)
                    
                    y_test = []
                    for unique_trajectory in unique_test_trajectories:
                        # Retrieve all instances with this trajectory id from full data set
                        test_trajectory_full = fold_full_df[fold_full_df[trajectory] == unique_trajectory]

                        # Split into numpy X and y
                        X_test_full = test_trajectory_full.drop(target, axis=1).to_numpy()
                        y_test.append(test_trajectory_full[target].to_numpy()[0])
                        del(test_trajectory_full)

                        # Get prediction of one trajectory via majority vote of all its inividual instance classifications
                        trajectory_prediction = np.bincount(learner.predict(X_test_full).astype(int)).argmax()
                        # append prediction to a list
                        predictions.append(trajectory_prediction)

                # Calculate and report our model's accuracy.
                model_accuracy = accuracy_score(y_test, predictions)
                model_macro_f1 = f1_score(y_test, predictions, average='macro')
                model_recall = recall_score(y_test, predictions, average='macro')
                model_precision = precision_score(y_test, predictions, average='macro', zero_division=0) # zero_division because sometimes no preictions of looping class
                model_auc = roc_auc_score(y_test, predictions, average = 'macro')

                # Save our model's performance for plotting.
                accuracy_history.append(model_accuracy)
                macro_f1_history.append(model_macro_f1)
                recall_history.append(model_recall)
                precision_history.append(model_precision)
                auc_history.append(model_auc)
                
                # Store feature importance.  For the time being, taking average feature importance of QBC members, although the differences between the members should be interesting
                if AL_METHOD != 5:
                    if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_cons.iat[execs * K + i, index+1] = abs(learner.estimator.coef_)[0]
                    else:
                        feat_importance_cons.iat[execs * K + i, index+1] = learner.estimator.feature_importances_
                    if SEPARATE_CLASSIFIER:
                            if query_estimator.estimator.__class__.__name__ == "SVC" or query_estimator.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_prod.iat[execs * K + i, index+1] = abs(query_estimator.estimator.coef_)[0]
                            else:
                                feat_importance_prod.iat[execs * K + i, index+1] = query_estimator.estimator.feature_importances_
                else:
                    if not SEPARATE_CLASSIFIER:
                        feat_importance_cons.iat[execs * K + i, index+1] = 0
                        for member in learner:
                            if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_cons.iat[execs * K + i, index+1] += abs(member.estimator.coef_)[0]
                            else:
                                feat_importance_cons.iat[execs * K + i, index+1] += member.estimator.feature_importances_
                            feat_importance_cons.iat[execs * K + i, index+1] = feat_importance_cons.iat[execs * K + i, index+1] / len(learner)
                    else:
                        if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_cons.iat[execs * K + i, index+1] = abs(learner.estimator.coef_)[0]
                        else:
                            feat_importance_cons.iat[execs * K + i, index+1] = learner.estimator.feature_importances_
                        
                        feat_importance_prod.iat[execs * K + i, index+1] = 0
                        for member in query_estimator:
                            if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_prod.iat[execs * K + i, index+1] += abs(member.estimator.coef_)[0]
                            else:
                                feat_importance_prod.iat[execs * K + i, index+1] += member.estimator.feature_importances_
                            feat_importance_prod.iat[execs * K + i, index+1] = feat_importance_prod.iat[execs * K + i, index+1] / len(query_estimator)
                
            # Save the class ratio over the cross validation
            selected_training_ratio = np.count_nonzero(selected_training_instance_labels == NUM_TARGET)/ (len(selected_training_instance_labels) - np.count_nonzero(selected_training_instance_labels == NUM_TARGET))
            series_ratios = pd.Series([fold_training_ratio, selected_training_ratio], index = balance_ratios.columns)
            balance_ratios = balance_ratios.append(series_ratios, ignore_index=True)

            # Save the specific labels chosen at each training point
            selected_instances_table = selected_instances_table.append(pd.Series(selected_training_instance_labels), ignore_index=True)

            # store results after each test
            accuracy_results = accuracy_results.append(pd.Series(accuracy_history), ignore_index=True)
            macro_f1_results = macro_f1_results.append(pd.Series(macro_f1_history), ignore_index=True)
            recall_results = recall_results.append(pd.Series(recall_history), ignore_index=True)
            precision_results = precision_results.append(pd.Series(precision_history), ignore_index=True)
            auc_results = auc_results.append(pd.Series(auc_history), ignore_index = True)

            if not TRAIN_ON_SUM:
                accuracy_results_single = accuracy_results_single.append(pd.Series(accuracy_history_single), ignore_index=True)
                macro_f1_results_single = macro_f1_results_single.append(pd.Series(macro_f1_history_single), ignore_index=True)
                recall_results_single = recall_results_single.append(pd.Series(recall_history_single), ignore_index=True)
                precision_results_single = precision_results_single.append(pd.Series(precision_history_single), ignore_index=True)
                auc_results_single = auc_results.append(pd.Series(auc_history_single), ignore_index = True)
            
    # Modify selected instances table so that only the positive class is a 1, the rest is 0. This makes the average value the selected class ratio
    # This is a problem if the target class is 0, because then all values will be set to 1.
    # Quick workaround of first setting all non-target values to 2, then setting these to 0 after setting the target values to 1. 
    # LIKELY BREAKS WHEN NO BINARY TARGET CLASS
    if NUM_TARGET != 0:
        selected_instances_table[selected_instances_table != NUM_TARGET] = 0
        selected_instances_table[selected_instances_table == NUM_TARGET] = 1
    else:
        selected_instances_table[selected_instances_table != NUM_TARGET] = 2
        selected_instances_table[selected_instances_table == NUM_TARGET] = 1
        selected_instances_table[selected_instances_table == 2] = 0
        
    all_results = list([macro_f1_results, recall_results, precision_results, selected_instances_table, auc_results])
    if not TRAIN_ON_SUM:
        all_results_single = list([macro_f1_results_single, recall_results_single, precision_results_single, selected_instances_table, auc_results_single])
    
    print("******************* FINISHED *******************")
    print("\n")
    print(balance_ratios.describe())
    
    # Set variables used for naming the results files
    if TRAIN_ON_SUM:
        train = "sum_"
    else: train = "full_"
    if SEPARATE_CLASSIFIER:
        sep_class = "sep_"
    else: sep_class = "same_"
    if OVERSAMPLE and UNDERSAMPLE:
        print("CAN'T OVER AND UNDERSAMPLE AT THE SAME TIME")
        raise()
    if OVERSAMPLE:
        samp_ratio = "OVER_" + str(SAMPLING_RATIO)
    elif UNDERSAMPLE:
        samp_ratio = "UNDER_" + str(SAMPLING_RATIO)
    else: samp_ratio = ""
    if AL_METHOD == 5:
        learners_qbc = '+'.join(str(x) for x in QBC_LEARNERS) + "_" + str(N_QBC_LEARNERS) + "_"
    else: learners_qbc = ""
    if AL_METHOD == 6 or AL_METHOD == 7:
        bdwidth = "BD_" + str(BANDWIDTH)
    else: bdwidth = ""
    if AL_METHOD == 1:
        weight = ''
    else: weight = "WF" + str(WEIGHTING_FACTOR) + "_"
    if OLD_DATA:
        old = "OLD_"
    else: old = ""
    if time_point == None:
        time_point_test = ""
    elif time_point == 'all': 
        time_point_test = 'all_time_points_'
    else: time_point_test = str(time_point) + "_"
        
    string = AL_switcher.get(AL_METHOD).__name__ + "_" + weight + old + bdwidth + learners_qbc + sep_class + train + "maj_" + samp_ratio + str(N_QUERIES) + "_" + time_point_test + "_new"
    measure_names = ['F1', 'Recall', 'Precision', "Selected Label Ratio", "AUC", "Feature Importance Consumer", "Feature Importance Producer"]
    measure_names = measure_names [:len(all_results)]
    
    plot_importance(feat_importance_cons, string, save = SAVEFIGS, cons = True)
    if SEPARATE_CLASSIFIER:
        plot_importance(feat_importance_prod, string, save = SAVEFIGS, cons = False)
    
    for result, measure_name in zip(all_results, measure_names):
        result.describe()
        plot_results(result, measure_name, False, string, save = SAVEFIGS)

    if not TRAIN_ON_SUM:
        string2 = AL_switcher.get(AL_METHOD).__name__ + "_"  + weight + old + bdwidth + learners_qbc + sep_class + train + "single_" + samp_ratio + str(N_QUERIES)
        for result, measure_name in zip(all_results_single, measure_names):
            result.describe()
            plot_results(result, measure_name, True, string2, save = SAVEFIGS)
    
    if SAVERATIO:
        np.savetxt("../Figures/" + string  + ".txt", balance_ratios.describe(), delimiter=', ', fmt='%1.4f')
        
    if SAVERESULTS and TRAIN_ON_SUM:
        for result, measure_name in zip(all_results, measure_names):
            result.to_pickle("../Results/" + string + "_" + measure_name + ".pkl")    
        trajectory_selection_frequencies.to_pickle("../Results/" + string + "_freq.pkl")
        
        feat_importance_cons.to_pickle("../Results/" + string + "_feat_imp_cons" + ".pkl")
        if SEPARATE_CLASSIFIER:
            feat_importance_prod.to_pickle("../Results/" + string + "_feat_imp_prod" + ".pkl")
            
    if SAVERESULTS and not TRAIN_ON_SUM:
        for result, measure_name in zip(all_results, measure_names):
            result.to_pickle("../Results/" + string + "_" + measure_name + ".pkl")    
        trajectory_selection_frequencies.to_pickle("../Results/" + string + "_freq.pkl")
        
        for result, measure_name in zip(all_results_single, measure_names):
            result.to_pickle("../Results/" + string2 + "_" + measure_name + ".pkl")   
        
        feat_importance_cons.to_pickle("../Results/" + string + "_feat_imp_cons" + ".pkl")
        if SEPARATE_CLASSIFIER:
            feat_importance_prod.to_pickle("../Results/" + string + "_feat_imp_prod" + ".pkl")

In [22]:
run_openml_test(data=X,k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

Starting execution:  1 / 3 - - - - - - - - 


NameError: name 'target' is not defined

In [1]:
# Running cross-validation of classifier and active learning approach. May be executed multiple times to combat the variability of performance at specific points during training
# Dataframes are converted to numpy arrays before using with modAL leaners because that is what they expect. These conversions are done throughout the process when necessary.
# The main dataframes (looping and looping_sum) are assumed to be globally set, the other settings correspond to those as defined above.
def run_test(k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, time_point = None, bandwidth_ = 0, pal_priors = 1, sc_pca_ = True):
    # Seed to allow different AL methods to train and test with the same splits and initialization sets over the course of the same number of executions with the same settings.
    np.random.seed(0)
    # Call the set_settings method to change settings as defined by the parameters of this method.
    set_settings(k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, bandwidth_, sc_pca_)
    
    # Keep track of results for each query. Used later to calculate incremental performance. +1 because the first value is the performance directly after initialization
    accuracy_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    macro_f1_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    recall_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    precision_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    auc_results = pd.DataFrame(columns = range(N_QUERIES + 1))

    # Keep track of results for each query. Used later to calculate incremental performance. +1 because the first value is the performance directly after initialization
    accuracy_results_single = pd.DataFrame(columns = range(N_QUERIES + 1))
    macro_f1_results_single = pd.DataFrame(columns = range(N_QUERIES + 1))
    recall_results_single = pd.DataFrame(columns = range(N_QUERIES + 1))
    precision_results_single = pd.DataFrame(columns = range(N_QUERIES + 1))
    auc_results_single = pd.DataFrame(columns = range(N_QUERIES + 1))
    
    # Keep track of feature importance during training. 
    feat_importance_cons = pd.DataFrame([], columns = range(N_QUERIES + 1), index = range(EXECUTIONS * K))
    if SEPARATE_CLASSIFIER:
        feat_importance_prod = pd.DataFrame([], columns = range(N_QUERIES + 1), index = range(EXECUTIONS * K))

    # Keep track of label ratio in data and selected label ratio
    balance_ratios = pd.DataFrame(columns = ["Training Folds Ratio", "Selected Training Instances Ratio"])
    selected_instances_table = pd.DataFrame(columns = range(N_QUERIES))

    # Keep track of selection frequencies of individual trajectories
    trajectory_selection_frequencies = pd.DataFrame(0, index=range(N_QUERIES), columns = looping_sum[trajectory])

    # Compute weighting factors for when instance weighting is selected.
    non_target_weight = np.unique(compute_sample_weight('balanced', looping_sum[target]), return_counts=True)[0][0] * (2-WEIGHTING_FACTOR)
    target_weight = np.unique(compute_sample_weight('balanced', looping_sum[target]), return_counts=True)[0][1] * WEIGHTING_FACTOR
    
    if time_point != None:
        # Convert hours to seconds when evaluating partial trajectories
        time_point = time_point * 3600
    
    # Perform K-fold validation for EXECUTIONS number of times. This gives more total repetitions, and thus a smoother view of learning performance
    for execs in range(EXECUTIONS):
        full_df = looping
        if time_point == None:
            df = looping_sum
        else:
            time_sepcific_idx = looping.groupby(["trip_id"]).apply(lambda x: abs(x["time_elapsed_s"] - time_point)).groupby(["trip_id"]).idxmin().apply(lambda x: x[1])
            df = looping.iloc[time_sepcific_idx].reset_index(drop=True)

        print("Starting execution: ", execs+1, "/" , EXECUTIONS,  "- - - - - - - - ")

        unique_classes = df[target].unique()
        n_labeled_examples = df.shape[0]

        if len(unique_classes) > INITIALIZATION_SET:
            print("INCREASE SIZE OF INITIALIZATION SET TO INCLUDE AT LEAST ONE INSTANCE OF EACH CLASS")
            raise
        
        # Make random selection for initialization set based on the sampling ratio and initialization set size
        n_minority = math.ceil(int(INITIALIZATION_SET * SAMPLING_RATIO))
        n_majority = INITIALIZATION_SET - n_minority
        
        initialization_indices = np.hstack([df[df[target]==NUM_TARGET].sample(n=n_minority, replace=False).index, \
                                                 df[df[target]!=NUM_TARGET].sample(n=n_majority, replace=False).index])
        initialization_set = df.iloc[initialization_indices]

        # Isolate the non-initialization examples to make up the sampling pool.
        df = df.drop(df.index[initialization_indices]).reset_index(drop=True)

        # Create separate set of initialization indices if using full trajectories for training
        if not TRAIN_ON_SUM:
            full_initialization_set = pd.DataFrame(columns = full_df.columns)

            for unique_trajectory in initialization_set[trajectory].unique():
                # Retrieve all instances with this trajectory id from full data set
                full_initialization_set = full_initialization_set.append(full_df[full_df[trajectory] == unique_trajectory])

            # Split into X and y
            X_initialize = full_initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = full_initialization_set[target].reset_index(drop=True)
            del(full_initialization_set)
        else:
            # Labeled training instances.
            X_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = initialization_set[target].reset_index(drop=True)
            
        # labeled training instances.
        X_aggr_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
        y_aggr_initialize = initialization_set[target].reset_index(drop=True)

        X_initialize[trajectory] = X_initialize[trajectory].apply(pd.to_numeric)
        y_initialize = pd.to_numeric(y_initialize)

        # Take stratified folds to make sure each fold contains enough instances of the majority class
        skf = StratifiedKFold(n_splits=K, shuffle = True)
        
        if time_point != None:
            pretime_df, ontime_df = time_specific_summarized_set(df, time_point, 30)
    
            X_aggr = ontime_df.drop(target, axis=1).to_numpy()
            y_aggr = ontime_df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))
            
        else:
            # Split aggregated instances into X and y
            X_aggr = df.drop(target, axis=1).to_numpy()
            y_aggr = df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))

        # K-fold cross validation
        for i in range(K):
            print("Validating on fold: ", i+1 , "/", K, end="\r")
            # Set train and test set for current fold
            if time_point != None:
                X_train = np.vstack([X_aggr[train_set_indices[i][0]], pretime_df.drop(target, axis=1).to_numpy()])
                y_train = np.hstack([y_aggr[train_set_indices[i][0]], pretime_df[target].to_numpy()])
        
                X_test = X_aggr[train_set_indices[i][1]]
                y_test = y_aggr[train_set_indices[i][1]]
        
            else: 
                X_train = X_aggr[train_set_indices[i][0]]
                y_train = y_aggr[train_set_indices[i][0]]
                
                X_test = X_aggr[train_set_indices[i][1]]
                y_test = y_aggr[train_set_indices[i][1]]
            
            # If the training set has fewer instances than the selected query budget, we have too few instances
            if len(X_train) < N_QUERIES:
                print("TOO FEW TRAINING INSTANCES TO TRAIN THIS FAR. REDUCE TO ", len(X_train))
                raise
            
            # This is why the trajectory identifier needs to be the first column. For creating the folds the dataframe needs to be turned into a numpy array,
            # but because of this the ability to select based on name is lost when we try to find these identifiers later.
            unique_train_trajectories = np.unique(X_train[:,0])
            unique_test_trajectories = np.unique(X_test[:,0])
            unique_initialization_trajectories = X_initialize[trajectory].unique()
            
            fold_full_df = full_df[full_df[trajectory].isin(np.concatenate([unique_initialization_trajectories, unique_train_trajectories, unique_test_trajectories]))]
            
            # Line only needed when testing without PCA and scaling, but doesn't do harm when not using PCA or scaling
            fold_X_initialize = X_initialize

            # Currently PCA commented out due to performance decrease for testing during the project. Can be uncommented to inlude PCA again
            if SC_PCA:
                # Initialization of SC and PCA:
                sc = StandardScaler()
                pca = PCA(n_components=0.9)

                # If training with all trajectory instance, need to scale and fit for this separately from the summarized dataset
                if not TRAIN_ON_SUM:
                    sc.fit(fold_full_df[fold_full_df[trajectory].isin(unique_train_trajectories)].drop(trajectory,axis=1).drop(target,axis=1))
                    fold_full_df = fold_full_df[[trajectory, target]].join(pd.DataFrame(sc.transform(fold_full_df.drop(trajectory,axis=1).drop(target,axis=1)), \
                                                                                            columns=fold_full_df.columns[2:], index=fold_full_df.index))
#                     pca.fit(fold_full_df[fold_full_df[trajectory].isin(unique_train_trajectories)].drop(trajectory,axis=1).drop(target,axis=1))
#                     fold_full_df = fold_full_df[[trajectory, target]].join(pd.DataFrame(pca.transform(fold_full_df.drop(trajectory,axis=1).drop(target,axis=1)), \
#                                                                                             columns=['PCA%i' %  i for i in range(len(pca.components_))], index=fold_full_df.index))

                    # if not using a separte classifier, need to apply same scaling and PCA of full dataset to the summarized one so training and predictions sets don't have different features
                    if SEPARATE_CLASSIFIER:
                        X_train[:,1:] = sc.transform(X_train[:,1:])
                        X_test[:,1:] = sc.transform(X_test[:,1:])
                        X_initialize_values = X_initialize.drop(trajectory,axis=1)
                        X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                        fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                         X_train = np.hstack((X_train[:,0,np.newaxis], pca.transform(X_train[:,1:])))
#                         X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                         X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                         X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                         fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
                    else:
                        X_train[:,1:] = sc.fit_transform(X_train[:,1:])
                        X_test[:,1:] = sc.transform(X_test[:,1:])
                        X_initialize_values = X_initialize.drop(trajectory,axis=1)
                        X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                        fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                         X_train = np.hstack((X_train[:,0,np.newaxis], pca.fit_transform(X_train[:,1:])))
#                         X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                         X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                         X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                         fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
                else:
                    X_train[:,1:] = sc.fit_transform(X_train[:,1:])
                    X_test[:,1:] = sc.transform(X_test[:,1:])
                    X_initialize_values = X_initialize.drop(trajectory,axis=1)
                    X_initialize_values = pd.DataFrame(sc.transform(X_initialize_values), columns=X_initialize_values.columns, index=X_initialize_values.index)
                    fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)

#                     X_train = np.hstack((X_train[:,0,np.newaxis], pca.fit_transform(X_train[:,1:])))
#                     X_test = np.hstack((X_test[:,0,np.newaxis], pca.transform(X_test[:,1:])))
#                     X_initialize_values = fold_X_initialize.drop(trajectory,axis=1)
#                     X_initialize_values = pd.DataFrame(pca.transform(X_initialize_values), columns=['PCA%i' %  i for i in range(len(pca.components_))], index=X_initialize_values.index)
#                     fold_X_initialize = pd.DataFrame(X_initialize[trajectory]).join(X_initialize_values)
            
            # Backup that allows for multiple ways of using the same y_test name but handling performance measures in different ways
            y_test_original = y_test

            if OVERSAMPLE and UNDERSAMPLE:
                print("CAN'T UNDER AND OVERSAMPLE AT THE SAME TIME")
                raise
            if OVERSAMPLE:
                sampler = RandomOverSampler(sampling_strategy=SAMPLING_RATIO)
                X_train, y_train = sampler.fit_resample(X_train, y_train)
            elif UNDERSAMPLE:
                sampler = RandomUnderSampler(sampling_strategy=SAMPLING_RATIO)
                X_train, y_train = sampler.fit_resample(X_train, y_train)

            # Save class ratio of instances in training fold
            fold_training_ratio = np.count_nonzero(y_train == NUM_TARGET)/np.count_nonzero(y_train != NUM_TARGET) # MOSTLY MAKES SENSE FOR BINARY TARGET CLASS

            model = ML_switcher.get(ML_METHOD)

            if AL_METHOD == 5 and not SEPARATE_CLASSIFIER: # Using QBC for main classifier, NOT RECOMMENDED WHEN COMPARING AL METHODS
                committee_list = []
                unique_trajectories_init = fold_X_initialize[trajectory].unique()
                qbc_initialization_size = math.ceil(INITIALIZATION_SET/(N_QBC_LEARNERS*len(QBC_LEARNERS))) + 1
                
                # For each specified QBC member, create N_QBC_LEARNERS learners to be added to the committee
                for qbc_model in QBC_LEARNERS:
                    # select random numbers, find trip id's for these in the aggr dataset. Then select all instances with these id's from the initilizing set and use them as new initializing set
                    for n in range(N_QBC_LEARNERS):
                        original_estimator = ML_switcher.get(qbc_model)
                        new_estimator = deepcopy(original_estimator)
                        
                        # The number of items for initializing the committe members is the proportion of number of members to size of the initialization set, rounded up.
                        # Considering the initialization set is usually small in this case due to low amounts of data, replacement is allowed which results insome overlap 
                        # in intialization sets make random selection for initialization set based on the sampling ratio and initialization set size
                        n_minority = math.ceil(int(qbc_initialization_size * SAMPLING_RATIO)) + 1
                        n_majority = qbc_initialization_size - n_minority

                        unique_negative_target = np.random.choice(initialization_set[initialization_set[target] != NUM_TARGET][trajectory].unique(),n_majority,replace=True)
                        unique_positive_target = np.random.choice(initialization_set[initialization_set[target] == NUM_TARGET][trajectory].unique(),n_minority,replace=True)
                        trajectories_init = np.concatenate((unique_positive_target, unique_negative_target), axis=None)
                        
                        idx_member_init = fold_X_initialize.index[fold_X_initialize[trajectory].isin(trajectories_init)].to_numpy()

                        X_member_init = fold_X_initialize.iloc[idx_member_init].to_numpy()
                        y_member_init = y_initialize.iloc[idx_member_init].to_numpy()

                        # Initializing learner
                        committee_member =  ActiveLearner(
                            estimator=new_estimator,
                            X_training=X_member_init, y_training=y_member_init
                        )
                        committee_list.append(committee_member)

                # Assembling the committee
                learner = Committee(
                    learner_list=committee_list,
                    query_strategy=QBC_STRATEGY
                )
            else: # NOT using QBC for main classifier. Should pretty much be always the case
                # Specify the core estimator along with its active learning model. Fit with the labeled data set
                learner = ActiveLearner(
                    estimator=deepcopy(model),
                    query_strategy=AL_switcher.get(AL_METHOD),
                    X_training=fold_X_initialize.to_numpy(), y_training=y_initialize.to_numpy()
                )

            # Create separate classifer for training and estimation
            if SEPARATE_CLASSIFIER:
                query_model = deepcopy(ML_switcher.get(ML_SEP_METHOD))
                new_query_strategy = deepcopy(AL_switcher.get(AL_METHOD))

                if AL_METHOD != 5:
                    query_estimator = ActiveLearner(
                            estimator=query_model,
                            query_strategy=new_query_strategy,
                            X_training=X_aggr_initialize.to_numpy(), y_training=y_aggr_initialize.to_numpy()
                    )
                else:
                    committee_list = []
                    unique_trajectories_init = X_aggr_initialize[trajectory].unique()
                    qbc_initialization_size = math.ceil(INITIALIZATION_SET/(N_QBC_LEARNERS*len(QBC_LEARNERS)))+1
                    
                    # For each specified QBC member, create N_QBC_LEARNERS learners to be added to the committee
                    for qbc_model in QBC_LEARNERS:
                        
                        # select random numbers, find trip id's for these in the aggr dataset. Then select all instances with these id's from the initilizing set and use them as new initializing set
                        for n in range(N_QBC_LEARNERS):
                            original_estimator = deepcopy(ML_switcher.get(qbc_model))
                            new_estimator = deepcopy(original_estimator)
                            
                            # make random selection for initialization set based on the sampling ratio and initialization set size
                            n_minority = math.ceil(int(qbc_initialization_size * SAMPLING_RATIO)) + 1
                            n_majority = qbc_initialization_size - n_minority

                            unique_negative_target = np.random.choice(initialization_set[initialization_set[target] != NUM_TARGET][trajectory].unique(),n_majority,replace=False)
                            unique_positive_target = np.random.choice(initialization_set[initialization_set[target] == NUM_TARGET][trajectory].unique(),n_minority,replace=False)
                            
                            trajectories_init = np.hstack([unique_positive_target, unique_negative_target])

                            idx_member_init = X_aggr_initialize[X_aggr_initialize[trajectory].isin(trajectories_init)].index.to_numpy()

                            X_member_init = X_aggr_initialize.iloc[idx_member_init].to_numpy()
                            y_member_init = y_aggr_initialize.iloc[idx_member_init].to_numpy()

                            # initializing learner
                            committee_member =  ActiveLearner(
                                estimator=new_estimator,
                                X_training=X_member_init, y_training=y_member_init
                            )
                            committee_list.append(committee_member)

                    # Assembling the committee
                    query_estimator = Committee(
                        learner_list=committee_list,
                        query_strategy=vote_entropy_sampling
                    )

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
            # Make first set of predictions (before querying)
            predictions = []

            if TRAIN_ON_SUM:
                predictions = learner.predict(X_test)

            else:
                # Training with all instances performance 1: Evaluating performance only for the summarizing instance (in this case single point at end of trajectory)
                predictions_single = learner.predict(X_test)

                unqueried_score_single = accuracy_score(y_test, predictions_single)
                unqueried_macro_f1_single = f1_score(y_test, predictions_single, average='macro')
                unqueried_recall_single = recall_score(y_test, predictions_single, average='macro')
                unqueried_precision_single = precision_score(y_test, predictions_single, average='macro', zero_division=0)
                unqueried_auc_single = roc_auc_score(y_test, predictions_single, average = 'macro')

                # Store performance of every calculated score
                accuracy_history_single = [unqueried_score_single]
                macro_f1_history_single = [unqueried_macro_f1_single]
                recall_history_single = [unqueried_recall_single]
                precision_history_single = [unqueried_precision_single]
                auc_history_single = [unqueried_auc_single]

                # Training with all instances performance 2: Evaluate performance by classifying all individual instances of each trajectory and taking a majority vote to get a 
                # prediction for that one trajectory.
                y_test = []
                for unique_trajectory in unique_test_trajectories:
                    # Retrieve all instances with this trajectory id from full data set
                    test_trajectory_full = fold_full_df[fold_full_df[trajectory] == unique_trajectory]
                    # Split into numpy X and y
                    X_test_full = test_trajectory_full.drop(target, axis=1).to_numpy()

                    y_test.append(test_trajectory_full[target].to_numpy()[0])
                    del(test_trajectory_full)

                    # get prediction of one trajectory via majority vote of all its inividual instance classifications
                    trajectory_prediction = np.bincount(learner.predict(X_test_full).astype(int)).argmax() # not sure why this has to be cast as int, already should be

                    # append prediction to a list
                    predictions.append(trajectory_prediction)
                del(X_test_full)

            # Record our learner's performance on the test data
            unqueried_score = accuracy_score(y_test, predictions)
            unqueried_macro_f1 = f1_score(y_test, predictions, average='macro')
            unqueried_recall = recall_score(y_test, predictions, average='macro')
            unqueried_precision = precision_score(y_test, predictions, average='macro', zero_division=0)
            unqueried_auc = roc_auc_score(y_test, predictions, average = 'macro')

            # Store performance of every calculated score
            accuracy_history = [unqueried_score]
            macro_f1_history = [unqueried_macro_f1]
            recall_history = [unqueried_recall]
            precision_history = [unqueried_precision]
            auc_history = [unqueried_auc]

            # Store feature importance.  For QBC, average importances of all committe members taken (although the differences between the members could be interesting)
            if AL_METHOD != 5:
                if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                    feat_importance_cons.iat[0,0] = abs(learner.estimator.coef_)[0]
                else:
                    feat_importance_cons.iat[0,0] = learner.estimator.feature_importances_
                if SEPARATE_CLASSIFIER:
                    if query_estimator.estimator.__class__.__name__ == "SVC" or query_estimator.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_prod.iat[0,0] = abs(query_estimator.estimator.coef_)[0]
                    else:
                        feat_importance_prod.iat[0,0] = query_estimator.estimator.feature_importances_
            else:
                if not SEPARATE_CLASSIFIER:    
                    feat_importance_cons.iat[0,0] = 0
                    for member in learner:
                        if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_cons.iat[0,0] += abs(member.estimator.coef_)[0]
                        else:
                            feat_importance_cons.iat[0,0] += member.estimator.feature_importances_
                        feat_importance_cons.iat[0,0] = feat_importance_cons.iat[0,0] / len(learner)
                else:
                    if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_cons.iat[0,0] = abs(learner.estimator.coef_)[0]
                    else:
                        feat_importance_cons.iat[0,0] = learner.estimator.feature_importances_

                    feat_importance_prod.iat[0,0] = 0
                    for member in query_estimator:
                        if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_prod.iat[0,0] += abs(member.estimator.coef_)[0]
                        else:
                            feat_importance_prod.iat[0,0] += member.estimator.feature_importances_
                        feat_importance_prod.iat[0,0] = feat_importance_prod.iat[0,0] / len(query_estimator)
            
            # Labels of selected training instances. Used for calculating metrics later on
            selected_training_instance_labels = np.array([])
            
            
            # For PAL: calculating gamma through bandwidth, with mean as default and using rbf as the kernel
            nominator = 2 * len(X_test) * len(X_test[0]) # number of instances, number of features
            denominator = (len(X_test) - 1) * np.log((len(X_test) - 1) / ((np.sqrt(2) * 10 ** -6) ** 2))
            bandwidth = np.sqrt(nominator / denominator)
            
            if BANDWIDTH == 0:
                bandwidth = np.sqrt(nominator / denominator)
            else: bandwidth = BANDWIDTH
            
            gamma = 0.5 * (bandwidth ** (-2))
            
            # Perform training and testing for the budget of N_QUERIES. The "unlabeled" dataset is queried for the most informative points according to our query strategy
            # which are consecutively added to the training set after which performance is directly measured.
            for index in range(N_QUERIES):
                # Setting up kernel with rbf as default and additional parameters (for PAL and xPAL)
                params = {}
                if AL_METHOD == 6 or AL_METHOD == 7 or AL_METHOD == 8 or AL_METHOD == 9:
                    S =  pairwise_kernels(X = X_train, Y = X_train, metric='rbf', gamma=gamma)
                    S = check_array(S, np.eye(len(X_train)))
                    if SEPARATE_CLASSIFIER:
                        params = {'n_classes': len(unique_classes), "labeled_instances": query_estimator.X_training, "labels": query_estimator.y_training, "kernel": S, "priors": pal_priors}
                    else: 
                        params = {'n_classes': len(unique_classes), "labeled_instances": learner.X_training, "labels": learner.y_training, "kernel": S, "priors": pal_priors}

                # Query instance using AL method
                if SEPARATE_CLASSIFIER:
                    query_index, query_instance = query_estimator.query(X_train, **params)
                else:
                    query_index, query_instance = learner.query(X_train, **params)

                # For measuring frequencies of selection of specific trajectories over the course of learning
                trajectory_selection_frequencies.iloc[index][int(X_train[query_index,0])] += 1
                
                # For measuring performance in terms of label selection
                selected_training_instance_labels = np.append(selected_training_instance_labels, y_train[query_index])

                # logic to make sure only do additional bootstrapping when using qbc, and only on the QBC classifiers
                boot_main = False
                boot_sep = False
#                 # This currently breaks. The bootstrapping takes random samples, but there are too few in the initialization set to guarantee at least one of each class
#                 if AL_METHOD == 5:
#                     if SEPARATE_CLASSIFIER:
#                         boot_main = False
#                         boot_sep = True
#                     else:
#                         boot_main = True
#                         boot_sep = True
                
                # Setting weighting of training instances when not randomly selecting them. Using two arrays guarantees that no overlapping values erase each other
                if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                    current_labels = np.append(learner.y_training, y_train[query_index].reshape(1, ))
                    sample_weights = np.zeros(len(current_labels)).astype(float)
                    temp_indices = np.where(current_labels == NUM_TARGET)
                    sample_weights[temp_indices] = target_weight
                    temp_indices = np.where(current_labels != NUM_TARGET)
                    sample_weights[temp_indices] = non_target_weight
                else:
                    sample_weights = None
    
        
                # Training the consumer with the summarizing queried instance
                if TRAIN_ON_SUM:
                    # Train our ActiveLearner model with the instance it has requested.
                    X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                    learner.teach(X=X_temp, y=y_temp, bootstrap=boot_main, sample_weight = sample_weights)

                    # Training a separate classifier (producer) with the summarizing queried instance
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap=boot_sep, sample_weight = sample_weights)

                # Training the model with all instances of queried trip
                else:
                    # Training a separate classifier (producer) used for selecting queries
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        
                        if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                            current_labels = np.append(query_estimator.y_training, y_train[query_index].reshape(1, ))
                            sample_weights = np.zeros(len(current_labels)).astype(float)
                            temp_indices = np.where(current_labels == NUM_TARGET)
                            sample_weights[temp_indices] = target_weight
                            temp_indices = np.where(current_labels != NUM_TARGET)
                            sample_weights[temp_indices] = non_target_weight
                        else:
                            sample_weights = None
                        
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap = boot_sep, sample_weight = sample_weights)

                    # Find trip_id of selected query in aggregated instances
                    query_trajectory_id = X_train[query_index,0]
                    # Retrieve all instances with this trajectory id from full data set

                    if not isinstance(query_trajectory_id, float):
                        query_trajectory_id = query_trajectory_id[0]
                    query_trajectory_full = fold_full_df[fold_full_df[trajectory] == query_trajectory_id] # for sep classifier the specific formatting returns one element in two dimensions

                    # Split into numpy X and y
                    X_train_full = query_trajectory_full.drop(target, axis=1).to_numpy()
                    y_train_full = query_trajectory_full[target].to_numpy()
                    del(query_trajectory_full)

                    if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                        current_labels = np.append(learner.y_training, y_train_full)
                        sample_weights = np.zeros(len(current_labels)).astype(float)
                        temp_indices = np.where(current_labels == NUM_TARGET)
                        sample_weights[temp_indices] = target_weight
                        temp_indices = np.where(current_labels != NUM_TARGET)
                        sample_weights[temp_indices] = 1 - (non_target_weight * 0.1)
                    else:
                        sample_weights = None
                    
                    # Add selected instances to training pool and teach the model re-train the model on the new set
                    # Train the consumer on either the full or 
                    learner.teach(X=X_train_full, y=y_train_full, bootstrap=boot_main, sample_weight = sample_weights)

                    del(X_train_full)
                    del(y_train_full)

                # Remove the queried instance from the unlabeled pool.
                X_train, y_train = np.delete(X_train, query_index, axis=0), np.delete(y_train, query_index)

                predictions = []
                # Evluate performance of
                if TRAIN_ON_SUM:
                    # Make predictions
                    predictions = learner.predict(X_test)

                # Evaluate performance on full trajectories by classifying all individual instances of each trajectory and taking a majority vote to get a prediction for that one trajectory
                else:
                    predictions_single = learner.predict(X_test)

                    # Calculate and report our model's accuracy for last instance.
                    model_accuracy_single = accuracy_score(y_test_original, predictions_single)
                    model_macro_f1_single = f1_score(y_test_original, predictions_single, average='macro')
                    model_recall_single = recall_score(y_test_original, predictions_single, average='macro')
                    model_precision_single = precision_score(y_test_original, predictions_single, average='macro', zero_division=0) # zero_division because sometimes no preictions of looping class
                    model_auc_single = roc_auc_score(y_test_original, predictions_single, average = 'macro')
                    
                    accuracy_history_single.append(model_accuracy_single)
                    macro_f1_history_single.append(model_macro_f1_single)
                    recall_history_single.append(model_recall_single)
                    precision_history_single.append(model_precision_single)  
                    auc_history_single.append(model_auc_single)
                    
                    y_test = []
                    for unique_trajectory in unique_test_trajectories:
                        # Retrieve all instances with this trajectory id from full data set
                        test_trajectory_full = fold_full_df[fold_full_df[trajectory] == unique_trajectory]

                        # Split into numpy X and y
                        X_test_full = test_trajectory_full.drop(target, axis=1).to_numpy()
                        y_test.append(test_trajectory_full[target].to_numpy()[0])
                        del(test_trajectory_full)

                        # Get prediction of one trajectory via majority vote of all its inividual instance classifications
                        trajectory_prediction = np.bincount(learner.predict(X_test_full).astype(int)).argmax()
                        # append prediction to a list
                        predictions.append(trajectory_prediction)

                # Calculate and report our model's accuracy.
                model_accuracy = accuracy_score(y_test, predictions)
                model_macro_f1 = f1_score(y_test, predictions, average='macro')
                model_recall = recall_score(y_test, predictions, average='macro')
                model_precision = precision_score(y_test, predictions, average='macro', zero_division=0) # zero_division because sometimes no preictions of looping class
                model_auc = roc_auc_score(y_test, predictions, average = 'macro')

                # Save our model's performance for plotting.
                accuracy_history.append(model_accuracy)
                macro_f1_history.append(model_macro_f1)
                recall_history.append(model_recall)
                precision_history.append(model_precision)
                auc_history.append(model_auc)
                
                # Store feature importance.  For the time being, taking average feature importance of QBC members, although the differences between the members should be interesting
                if AL_METHOD != 5:
                    if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                        feat_importance_cons.iat[execs * K + i, index+1] = abs(learner.estimator.coef_)[0]
                    else:
                        feat_importance_cons.iat[execs * K + i, index+1] = learner.estimator.feature_importances_
                    if SEPARATE_CLASSIFIER:
                            if query_estimator.estimator.__class__.__name__ == "SVC" or query_estimator.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_prod.iat[execs * K + i, index+1] = abs(query_estimator.estimator.coef_)[0]
                            else:
                                feat_importance_prod.iat[execs * K + i, index+1] = query_estimator.estimator.feature_importances_
                else:
                    if not SEPARATE_CLASSIFIER:
                        feat_importance_cons.iat[execs * K + i, index+1] = 0
                        for member in learner:
                            if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_cons.iat[execs * K + i, index+1] += abs(member.estimator.coef_)[0]
                            else:
                                feat_importance_cons.iat[execs * K + i, index+1] += member.estimator.feature_importances_
                            feat_importance_cons.iat[execs * K + i, index+1] = feat_importance_cons.iat[execs * K + i, index+1] / len(learner)
                    else:
                        if learner.estimator.__class__.__name__ == "SVC" or learner.estimator.__class__.__name__ == "LogisticRegression":
                            feat_importance_cons.iat[execs * K + i, index+1] = abs(learner.estimator.coef_)[0]
                        else:
                            feat_importance_cons.iat[execs * K + i, index+1] = learner.estimator.feature_importances_
                        
                        feat_importance_prod.iat[execs * K + i, index+1] = 0
                        for member in query_estimator:
                            if member.estimator.__class__.__name__ == "SVC" or member.estimator.__class__.__name__ == "LogisticRegression":
                                feat_importance_prod.iat[execs * K + i, index+1] += abs(member.estimator.coef_)[0]
                            else:
                                feat_importance_prod.iat[execs * K + i, index+1] += member.estimator.feature_importances_
                            feat_importance_prod.iat[execs * K + i, index+1] = feat_importance_prod.iat[execs * K + i, index+1] / len(query_estimator)
                
            # Save the class ratio over the cross validation
            selected_training_ratio = np.count_nonzero(selected_training_instance_labels == NUM_TARGET)/ (len(selected_training_instance_labels) - np.count_nonzero(selected_training_instance_labels == NUM_TARGET))
            series_ratios = pd.Series([fold_training_ratio, selected_training_ratio], index = balance_ratios.columns)
            balance_ratios = balance_ratios.append(series_ratios, ignore_index=True)

            # Save the specific labels chosen at each training point
            selected_instances_table = selected_instances_table.append(pd.Series(selected_training_instance_labels), ignore_index=True)

            # store results after each test
            accuracy_results = accuracy_results.append(pd.Series(accuracy_history), ignore_index=True)
            macro_f1_results = macro_f1_results.append(pd.Series(macro_f1_history), ignore_index=True)
            recall_results = recall_results.append(pd.Series(recall_history), ignore_index=True)
            precision_results = precision_results.append(pd.Series(precision_history), ignore_index=True)
            auc_results = auc_results.append(pd.Series(auc_history), ignore_index = True)

            if not TRAIN_ON_SUM:
                accuracy_results_single = accuracy_results_single.append(pd.Series(accuracy_history_single), ignore_index=True)
                macro_f1_results_single = macro_f1_results_single.append(pd.Series(macro_f1_history_single), ignore_index=True)
                recall_results_single = recall_results_single.append(pd.Series(recall_history_single), ignore_index=True)
                precision_results_single = precision_results_single.append(pd.Series(precision_history_single), ignore_index=True)
                auc_results_single = auc_results.append(pd.Series(auc_history_single), ignore_index = True)
            
    # Modify selected instances table so that only the positive class is a 1, the rest is 0. This makes the average value the selected class ratio
    # This is a problem if the target class is 0, because then all values will be set to 1.
    # Quick workaround of first setting all non-target values to 2, then setting these to 0 after setting the target values to 1. 
    # LIKELY BREAKS WHEN NO BINARY TARGET CLASS
    if NUM_TARGET != 0:
        selected_instances_table[selected_instances_table != NUM_TARGET] = 0
        selected_instances_table[selected_instances_table == NUM_TARGET] = 1
    else:
        selected_instances_table[selected_instances_table != NUM_TARGET] = 2
        selected_instances_table[selected_instances_table == NUM_TARGET] = 1
        selected_instances_table[selected_instances_table == 2] = 0
        
    all_results = list([macro_f1_results, recall_results, precision_results, selected_instances_table, auc_results])
    if not TRAIN_ON_SUM:
        all_results_single = list([macro_f1_results_single, recall_results_single, precision_results_single, selected_instances_table, auc_results_single])
    
    print("******************* FINISHED *******************")
    print("\n")
    print(balance_ratios.describe())
    
    # Set variables used for naming the results files
    if TRAIN_ON_SUM:
        train = "sum_"
    else: train = "full_"
    if SEPARATE_CLASSIFIER:
        sep_class = "sep_"
    else: sep_class = "same_"
    if OVERSAMPLE and UNDERSAMPLE:
        print("CAN'T OVER AND UNDERSAMPLE AT THE SAME TIME")
        raise()
    if OVERSAMPLE:
        samp_ratio = "OVER_" + str(SAMPLING_RATIO)
    elif UNDERSAMPLE:
        samp_ratio = "UNDER_" + str(SAMPLING_RATIO)
    else: samp_ratio = ""
    if AL_METHOD == 5:
        learners_qbc = '+'.join(str(x) for x in QBC_LEARNERS) + "_" + str(N_QBC_LEARNERS) + "_"
    else: learners_qbc = ""
    if AL_METHOD == 6 or AL_METHOD == 7:
        bdwidth = "BD_" + str(BANDWIDTH)
    else: bdwidth = ""
    if AL_METHOD == 1:
        weight = ''
    else: weight = "WF" + str(WEIGHTING_FACTOR) + "_"
    if OLD_DATA:
        old = "OLD_"
    else: old = ""
    if time_point == None:
        time_point_test = ""
    elif time_point == 'all': 
        time_point_test = 'all_time_points_'
    else: time_point_test = str(time_point) + "_"
        
    string = AL_switcher.get(AL_METHOD).__name__ + "_" + weight + old + bdwidth + learners_qbc + sep_class + train + "maj_" + samp_ratio + str(N_QUERIES) + "_" + time_point_test + "_new"
    measure_names = ['F1', 'Recall', 'Precision', "Selected Label Ratio", "AUC", "Feature Importance Consumer", "Feature Importance Producer"]
    measure_names = measure_names [:len(all_results)]
    
    plot_importance(feat_importance_cons, string, save = SAVEFIGS, cons = True)
    if SEPARATE_CLASSIFIER:
        plot_importance(feat_importance_prod, string, save = SAVEFIGS, cons = False)
    
    for result, measure_name in zip(all_results, measure_names):
        result.describe()
        plot_results(result, measure_name, False, string, save = SAVEFIGS)

    if not TRAIN_ON_SUM:
        string2 = AL_switcher.get(AL_METHOD).__name__ + "_"  + weight + old + bdwidth + learners_qbc + sep_class + train + "single_" + samp_ratio + str(N_QUERIES)
        for result, measure_name in zip(all_results_single, measure_names):
            result.describe()
            plot_results(result, measure_name, True, string2, save = SAVEFIGS)
    
    if SAVERATIO:
        np.savetxt("../Figures/" + string  + ".txt", balance_ratios.describe(), delimiter=', ', fmt='%1.4f')
        
    if SAVERESULTS and TRAIN_ON_SUM:
        for result, measure_name in zip(all_results, measure_names):
            result.to_pickle("../Results/" + string + "_" + measure_name + ".pkl")    
        trajectory_selection_frequencies.to_pickle("../Results/" + string + "_freq.pkl")
        
        feat_importance_cons.to_pickle("../Results/" + string + "_feat_imp_cons" + ".pkl")
        if SEPARATE_CLASSIFIER:
            feat_importance_prod.to_pickle("../Results/" + string + "_feat_imp_prod" + ".pkl")
            
    if SAVERESULTS and not TRAIN_ON_SUM:
        for result, measure_name in zip(all_results, measure_names):
            result.to_pickle("../Results/" + string + "_" + measure_name + ".pkl")    
        trajectory_selection_frequencies.to_pickle("../Results/" + string + "_freq.pkl")
        
        for result, measure_name in zip(all_results_single, measure_names):
            result.to_pickle("../Results/" + string2 + "_" + measure_name + ".pkl")   
        
        feat_importance_cons.to_pickle("../Results/" + string + "_feat_imp_cons" + ".pkl")
        if SEPARATE_CLASSIFIER:
            feat_importance_prod.to_pickle("../Results/" + string + "_feat_imp_prod" + ".pkl")
        

In [31]:
# Method for plotting the saved results as figures. 
def plot_results(results, measure_name, single, name, save = False):
    mean_performance = results.describe().loc[['mean'], :].to_numpy()[0]
    lower_quartiles = results.describe().loc[['25%'], :].to_numpy()[0]
    upper_quartiles = results.describe().loc[['75%'], :].to_numpy()[0]
    medians = results.describe().loc[['50%'], :].to_numpy()[0]

    quartiles = list(zip(lower_quartiles,upper_quartiles))

    # Plot our performance over time.
    fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)

    ax.plot(mean_performance)
    ax.scatter(range(len(mean_performance)), mean_performance, s=13,c="blue")

    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=5, integer=True))
    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=10))
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))

    if measure_name == "Selected Label Ratio":
        ax.set_ylim(bottom=0, top = 1, auto=False)
    else:
        	ax.set_ylim(bottom=0.4, top = 1, auto=False) # bottom=0, top=1
    ax.grid(True)

    ax.set_title('Incremental Classification ' + measure_name + " " + AL_switcher.get(AL_METHOD).__name__)
    ax.set_xlabel('Query iteration')
    ax.set_ylabel('Classification ' + measure_name)
#     ax.legend(loc="lower right")

    if measure_name == 'AUC':
        x = range(len(mean_performance))
        x = x[1:]
        y = mean_performance
        rand_predict = mean_performance[0]
        y = mean_performance[1:]
        
        x = np.log2(x)
        A=trapz(y)
        Arand = rand_predict * x[-1]
        Amax = x[-1]
        
        global_score = (A-Arand)/(Amax-Arand)
        
        print("ALC is: ", global_score)
        
        if save:
            string = "../Figures/" + name + "_" + "ALC" + ".txt"
            text_file = open(string, "w")
            n = text_file.write(str(global_score))
            text_file.close()

    if measure_name != "Selected Label Ratio":
        plt.plot((range(len(mean_performance)),range(len(mean_performance))),([i for (i,j) in quartiles], [j for (i,j) in quartiles]),c='black')
        plt.plot(range(len(mean_performance)), [i for (i,j) in quartiles], '_', markersize = 6,c='blue')
        plt.plot(range(len(mean_performance)), [j for (i,j) in quartiles], '_', markersize = 6,c='blue')

    fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                        hspace=0.4, wspace=0.3)
    if save:
        string = "../Figures/" + name + "_" + measure_name + ".png"
        plt.savefig(string, bbox_inches='tight')
    plt.show()

In [32]:
# Method for plotting feature importances over the course of training for the different sizes of sliding windows and the base features.
def plot_importance(importances, string, save = False, cons=True):
    timeframes = ["base", "expanding", "10 minutes", "60 minutes"]
    timeframe_start_indices = [1, 7, 39, 71, len(importances.iloc[0,0])]
    for frame_name, frame_num in zip(timeframes, range(len(timeframes))):
        feature_names = looping_sum.columns[looping_sum.columns != target]
        feature_names = feature_names[timeframe_start_indices[frame_num]:timeframe_start_indices[frame_num+1]-1]
        plt.figure()
        fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)
        colors = cm.rainbow(np.linspace(0,1,len(feature_names))) # gist_ncar

        i = -1
        for feat, color in zip(range(timeframe_start_indices[frame_num],timeframe_start_indices[frame_num+1]-1), colors):
            i += 1
            one_feat_importance = importances.applymap(lambda x: x[feat], na_action = 'ignore')
            feature_name = feature_names[i]
            
            mean_importance = one_feat_importance.describe().loc[['mean'], :].to_numpy()[0]
            
            # Interquartile ranges not shown here due to messy graphs, but this can simply be uncommented.
    #         lower_quartiles = one_feat_importance.describe().loc[['25%'], :].to_numpy()[0]
    #         upper_quartiles = one_feat_importance.describe().loc[['75%'], :].to_numpy()[0]
    #         medians = one_feat_importance.describe().loc[['50%'], :].to_numpy()[0]
    
            ax.plot(mean_importance,c=color)
            ax.scatter(range(len(mean_importance)), mean_importance, s=13, marker = "")
            ax.annotate(xy=(range(len(mean_importance))[-1], mean_importance[-1]), xytext=(5,0), textcoords='offset points', text=feature_name, va='center', size = 7)

        title = string.split("_")[0] + "_" + string.split("_")[1] + "_" + string.split("_")[2]

        ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=5, integer=True))
        ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=10))
        ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))

    #     ax.set_ylim(bottom=0, top=1)
        ax.set_ylim(auto=True)
        ax.grid(True)

        ax.set_title('Feature Importances ' + title + "_" + timeframes[frame_num])
        ax.set_xlabel('Query iteration')
        ax.set_ylabel('Feature Importance ')
        ax.legend(feature_names, bbox_to_anchor=(1.2, 1))
        
        fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                            hspace=0.4, wspace=0.3)

        if save:
            if cons:
                cons_or_prod = "cons_"
            else:
                cons_or_prod = "prod_"
            filename = "../Figures/" + string + "_" + cons_or_prod + timeframes[frame_num] + ".png"
            plt.savefig(filename, bbox_inches='tight')
        plt.show()

In [33]:
%%time
# # # random sampling on summarized dataset
# set_settings(k_ = 5, execs_ = 20, n_queries_ = 40, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # uncertainty sampling on summarized dataset
# set_settings(k_ = 5, execs_ = 1, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # Density sampling on summarized dataset
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # PAL sampling on summarized dataset
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # random sampling on full dataset
# set_settings(k_ = 5, execs_ = 2, n_queries_ = 100, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # uncertainty sampling on full dataset
# set_settings(k_ = 5, execs_ = 2, n_queries_ = 100, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # Density sampling on full dataset
# set_settings(k_ = 5, execs_ = 2, n_queries_ = 100, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # # # PAL sampling on full dataset
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()





# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 5, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 5, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [5], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [5], n_qbc_learners_ = 4, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [5], n_qbc_learners_ = 5, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 1, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 2, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,5], n_qbc_learners_ = 1, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,5], n_qbc_learners_ = 2, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,5], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,5], n_qbc_learners_ = 1, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,5], n_qbc_learners_ = 2, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,5], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,5], n_qbc_learners_ = 1, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,5], n_qbc_learners_ = 2, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,5], n_qbc_learners_ = 3, save_figs_ = True, save_ratio_ = True)
# run_test()
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,5], n_qbc_learners_ = 5, save_figs_ = True, save_ratio_ = True)
# run_test()





# test_weights = [0, 0.2, 0.4, 0.6, 0.8, 1]
# # # uncertainty sampling on summarized dataset
# for test_weight in test_weights:
#     set_settings(k_ = 5, execs_ = 20, n_queries_ = 40, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#                  ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], weighting_factor_ = test_weight, n_qbc_learners_ = 4, \
#                  save_figs_ = True, save_ratio_ = True, save_results_ = True)
#     run_test()

# set_settings(k_ = 5, execs_ = 20, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], weighting_factor_ = 0, n_qbc_learners_ = 4, \
#              save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# test_weights = [0, 0.2, 0.4, 0.6, 0.8, 1]
# # # pal sampling on summarized dataset
# for test_weight in test_weights:
#     set_settings(k_ = 5, execs_ = 20, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#                  ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], weighting_factor_ = test_weight, n_qbc_learners_ = 4, \
#                  save_figs_ = True, save_ratio_ = True, save_results_ = True)
#     run_test()

# test_weights = [0, 0.2, 0.4, 0.6, 0.8, 1]
# # # xpal sampling on summarized dataset
# for test_weight in test_weights:
#     set_settings(k_ = 5, execs_ = 20, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#                  ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], weighting_factor_ = test_weight, n_qbc_learners_ = 4, \
#                  save_figs_ = True, save_ratio_ = True, save_results_ = True)
#     run_test()

# test_weights = [0, 0.2, 0.4, 0.6, 0.8, 1]
# # # density sampling on summarized dataset
# for test_weight in test_weights:
#     set_settings(k_ = 5, execs_ = 20, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#                  ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], weighting_factor_ = test_weight, n_qbc_learners_ = 4, \
#                  save_figs_ = True, save_ratio_ = True, save_results_ = True)
#     run_test()

# Quick testing
# set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 5, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0.8, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test()

# # bandwidth testing
# bandwidths = [0.01, 1.5, 2, 2.5, 3, 3.5, 10] # doesn't seem to change anything at all
# for band in bandwidths:
#     set_settings(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#                  ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#                  weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, bandwidth_ = band)
#     run_test()




# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)
# run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True)





# # # Testing performance at different time points for random sampling
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# # run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
# #              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
# #              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = "all")

# # # Testing performance at different time points for uncertainty sampling
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)

# run_test(k_ = 5, execs_ = 10, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = "all")

# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = 20, pal_priors = 1)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = None, pal_priors = [0.5,10])
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = None, pal_priors = [10,0.5]) #  best results [10,0.5]

# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = None, bandwidth_ = 0.1, pal_priors = [10,0.5]) #  best results [10,0.5]
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = None, bandwidth_ = 0.3, pal_priors = [10,0.5]) #  best results [10,0.5]

# run_test(k_ = 5, execs_ = 1, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = False, save_ratio_ = False, save_results_ = False, time_point = 25) #  best results [10,0.5]

# # # Testing performance at different time points for random sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 1, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# # # Testing performance at different time points for uncertainty sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 2, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# # # Testing performance at different time points for density-weighted sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

run_test(k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# # # Testing performance at different time points for qbc sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)

run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)

# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,1], n_qbc_learners_ = 8, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# # # Testing performance at different time points for pal sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 6, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# # # Testing performance at different time points for xpal sampling
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 10)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 16)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 20)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 25)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 30)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 35)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = 40)
# run_test(k_ = 5, execs_ = 30, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = False, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 7, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)

# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 5, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 6, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 7, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2], n_qbc_learners_ = 8, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 5, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 6, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 7, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3], n_qbc_learners_ = 8, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 5, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 6, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 7, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [1], n_qbc_learners_ = 8, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 1, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,1], n_qbc_learners_ = 1, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,1], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,1], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,1], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,1], n_qbc_learners_ = 1, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,1], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,1], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [3,1], n_qbc_learners_ = 4, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,1], n_qbc_learners_ = 1, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,1], n_qbc_learners_ = 2, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)
# run_test(k_ = 5, execs_ = 5, n_queries_ = 100, n_init_ = 10, train_sum_ = True, sep_class_ = True, \
#              ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 5, qbc_learners_ = [2,3,1], n_qbc_learners_ = 3, \
#              weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)


NameError: name 'looping_sum' is not defined